In [1]:
!pip install -e .

Obtaining file:///Users/matthewdeane/Documents/Data%20Science/python/_projects/py-tidymodels/_md
ERROR: file:///Users/matthewdeane/Documents/Data%20Science/python/_projects/py-tidymodels/_md does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# py-tidymodels imports
from py_workflows import workflow
from py_parsnip import linear_reg
from py_rsample import vfold_cv, initial_split, training, testing
from py_yardstick import metric_set, rmse, mae, r_squared, smape
from py_workflowsets import WorkflowSet

from py_recipes import recipe
from py_recipes.selectors import all_numeric, all_nominal, starts_with, ends_with

from py_visualize import plot_forecast, plot_residuals

# Set random seed for reproducibility
np.random.seed(42)

# Import Data

In [3]:
# import raw data
raw_data = pd.read_csv('__data/preem.csv')

In [4]:
df = raw_data

# set date as index
#df = raw_data.set_index('date')

# convert date to datetime
df['date'] = pd.to_datetime(df['date'])
display(df)

,date,mean_med_diesel_crack_input1_trade_month_lag2,mean_nwe_hsfo_crack_trade_month_lag1,mean_nwe_lsfo_crack_trade_month,mean_nwe_ulsfo_crack_trade_month_lag3,mean_sing_gasoline_vs_vlsfo_trade_month,mean_sing_vlsfo_crack_trade_month_lag3,new_sweet_sr_margin,target,totaltar
0,2020-04-01,-56.18,-11.43,-3.07,28.58,-16.09,26.91,0.47,137.65,0.00
1,2020-05-01,-42.36,-9.17,3.89,22.64,-8.75,20.36,0.57,113.53,0.00
2,2020-06-01,-30.21,-8.58,-3.44,13.43,-4.28,11.32,6.61,43.31,0.00
3,2020-07-01,-28.86,-6.86,-2.71,10.74,-3.20,8.53,-1.55,79.77,0.00
4,2020-08-01,-38.80,-5.86,-2.63,4.58,-5.35,6.26,-2.70,48.27,0.00
5,2020-09-01,-41.76,-5.62,-2.08,7.69,-3.26,6.15,-2.10,98.01,0.00
6,2020-10-01,-43.99,-5.10,0.16,8.09,-4.31,5.80,-3.34,49.77,0.00
7,2020-11-01,-42.94,-5.43,1.80,7.86,-7.40,6.99,-2.53,35.48,0.00
8,2020-12-01,-41.55,-3.39,0.94,6.68,-7.65,6.22,0.00,63.98,0.00
9,2021-01-01,-41.18,-6.03,0.63,8.67,-7.67,8.22,-0.05,109.76,0.00


# py-rsample

In [5]:
from py_rsample import initial_time_split, time_series_cv

In [6]:
# Create train/test split
split = initial_split(df, prop=0.75, seed=123)
train_data = training(split)
test_data = testing(split)

print(f"Training set: {train_data.shape[0]} rows")
print(f"Test set: {test_data.shape[0]} rows")

Training set: 42 rows
Test set: 15 rows


In [7]:
# Example: Train on 2020, test on first half of 2021
split_absolute = initial_time_split(
    df,
    date_column="date",
    train_start="2022-01-01",
    train_end="2023-12-01",
    test_start="2024-01-01",
    test_end="2024-06-01"
)

train_abs = split_absolute.training()
test_abs = split_absolute.testing()

print(f"Training: {len(train_abs)} days from {train_abs['date'].min()} to {train_abs['date'].max()}")
print(f"Testing:  {len(test_abs)} days from {test_abs['date'].min()} to {test_abs['date'].max()}")

# bug - thinks its days not months (monthly data)

Training: 24 days from 2022-01-01 00:00:00 to 2023-12-01 00:00:00
Testing:  6 days from 2024-01-01 00:00:00 to 2024-06-01 00:00:00


In [8]:
# Example 1: Train on first 18 months, test on last 6 months
split_relative1 = initial_time_split(
    df,
    date_column="date",
    train_start="start",
    train_end="start + 18 months",
    test_start="end - 6 months",
    test_end="end"
)

train_rel1 = split_relative1.training()
test_rel1 = split_relative1.testing()

print("Example 1: First 18 months vs last 6 months")
print(f"Training: {len(train_rel1)} days from {train_rel1['date'].min()} to {train_rel1['date'].max()}")
print(f"Testing:  {len(test_rel1)} days from {test_rel1['date'].min()} to {test_rel1['date'].max()}")
print()

# bug - thinks its days not months (monthly data)

Example 1: First 18 months vs last 6 months
Training: 18 days from 2020-04-01 00:00:00 to 2021-09-01 00:00:00
Testing:  6 days from 2024-07-01 00:00:00 to 2024-12-01 00:00:00



In [9]:
# Train on specific year (2020), test on last 3 months of available data
split_mixed = initial_time_split(
    df,
    date_column="date",
    train_start="start",
    train_end="2023-12-01",
    test_start="end - 12 months",
    test_end="end"
)

train_mix = split_mixed.training()
test_mix = split_mixed.testing()

print("Mixed: Train on 2020, test on last 90 days of data")
print(f"Training: {len(train_mix)} days from {train_mix['date'].min()} to {train_mix['date'].max()}")
print(f"Testing:  {len(test_mix)} days from {test_mix['date'].min()} to {test_mix['date'].max()}")

# bug - thinks its days not months (monthly data)

Mixed: Train on 2020, test on last 90 days of data
Training: 45 days from 2020-04-01 00:00:00 to 2023-12-01 00:00:00
Testing:  12 days from 2024-01-01 00:00:00 to 2024-12-01 00:00:00


# 1. Single Model Fitting

In [10]:
FORMULA_STR = "target ~ totaltar +mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1"

## 1.1 - Prophet

In [11]:
# SINGLE MODEL fitting
from py_visualize import plot_forecast
from py_parsnip import prophet_reg, arima_reg, rand_forest, linear_reg

# Create Prophet specification
# Prophet will automatically detect weekly seasonality in our data
spec_prophet = prophet_reg(
    n_changepoints=25,  # Number of potential trend changes
    changepoint_prior_scale=0.05,  # Flexibility of trend
    seasonality_prior_scale=10.0   # Flexibility of seasonality
)

print(spec_prophet)

# # Fit Prophet - FAILING
fit_prophet = spec_prophet.fit(train_data, FORMULA_STR)
print("Prophet model fitted!")

# Evaluate and extract outputs
fit_prophet = fit_prophet.evaluate(test_data)
outputs_prophet, coefs_prophet, stats_prophet = fit_prophet.extract_outputs()

display(outputs_prophet)
display(coefs_prophet)
display(stats_prophet)

# Create interactive forecast plot
fig = plot_forecast(fit_prophet, title="Sales Forecast")
fig.show()

fig = plot_residuals(fit_prophet, title="Sales Forecast")
fig.show()

00:29:13 - cmdstanpy - INFO - Chain [1] start processing


ModelSpec(model_type='prophet_reg', engine='prophet', mode='regression', args={'growth': 'linear', 'changepoint_prior_scale': 0.05, 'seasonality_prior_scale': 10.0, 'seasonality_mode': 'additive', 'n_changepoints': 25, 'changepoint_range': 0.8, 'yearly_seasonality': 'auto', 'weekly_seasonality': 'auto', 'daily_seasonality': 'auto'}, date_col=None)


00:29:13 - cmdstanpy - INFO - Chain [1] done processing


Prophet model fitted!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,112.812329,137.65,24.837671,train,prophet_reg,,global
1,2020-05-01,113.53,115.589899,113.53,-2.059899,train,prophet_reg,,global
2,2020-06-01,43.31,66.349488,43.31,-23.039488,train,prophet_reg,,global
3,2020-07-01,79.77,70.154397,79.77,9.615603,train,prophet_reg,,global
4,2020-08-01,48.27,64.104879,48.27,-15.834879,train,prophet_reg,,global
5,2020-09-01,98.01,118.531915,98.01,-20.521915,train,prophet_reg,,global
6,2020-10-01,49.77,79.545138,49.77,-29.775138,train,prophet_reg,,global
7,2020-11-01,35.48,58.856986,35.48,-23.376986,train,prophet_reg,,global
8,2020-12-01,63.98,99.834320,63.98,-35.854320,train,prophet_reg,,global
9,2021-01-01,109.76,115.407468,109.76,-5.647468,train,prophet_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,growth,linear,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
1,changepoint_prior_scale,0.05,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
2,seasonality_prior_scale,10.0,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
3,seasonality_mode,additive,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
4,n_changepoints,25,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,21.537087,train,prophet_reg,,global
1,mae,17.262224,train,prophet_reg,,global
2,mape,19.910901,train,prophet_reg,,global
3,smape,18.557042,train,prophet_reg,,global
4,r_squared,0.626847,train,prophet_reg,,global
5,mda,80.487805,train,prophet_reg,,global
6,rmse,45.21693,test,prophet_reg,,global
7,mae,38.048444,test,prophet_reg,,global
8,mape,29.854629,test,prophet_reg,,global
9,smape,32.221975,test,prophet_reg,,global


## 1.2 - ARIMA

In [12]:
# Create ARIMA specification
spec_arima = arima_reg(
    seasonal_period=7,  # Weekly seasonality
    non_seasonal_ar=1,
    non_seasonal_differences=1,
    non_seasonal_ma=1,
    seasonal_ar=1,
    seasonal_differences=0,
    seasonal_ma=1
)

print(spec_arima)

# Fit ARIMA
fit_arima = spec_arima.fit(train_data, FORMULA_STR)
print("ARIMA model fitted!")

# Predict on test data
pred_arima = fit_arima.predict(test_data)

# Evaluate and extract outputs
fit_arima = fit_arima.evaluate(test_data)
outputs_arima, coefs_arima, stats_arima = fit_arima.extract_outputs()

display(outputs_arima)
display(coefs_arima)
display(stats_arima)

# Create interactive forecast plot
fig = plot_forecast(fit_arima, title="Sales Forecast", prediction_intervals=True)
fig.show()

ModelSpec(model_type='arima_reg', engine='statsmodels', mode='regression', args={'seasonal_period': 7, 'non_seasonal_ar': 1, 'non_seasonal_differences': 1, 'non_seasonal_ma': 1, 'seasonal_ar': 1, 'seasonal_differences': 0, 'seasonal_ma': 1}, date_col=None)
ARIMA model fitted!


/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.



,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,5.250682,137.65,132.399318,train,arima_reg,,global
1,2020-05-01,113.53,150.527218,113.53,-36.997218,train,arima_reg,,global
2,2020-06-01,43.31,107.848484,43.31,-64.538484,train,arima_reg,,global
3,2020-07-01,79.77,28.240591,79.77,51.529409,train,arima_reg,,global
4,2020-08-01,48.27,88.007526,48.27,-39.737526,train,arima_reg,,global
5,2020-09-01,98.01,41.646030,98.01,56.363970,train,arima_reg,,global
6,2020-10-01,49.77,108.570545,49.77,-58.800545,train,arima_reg,,global
7,2020-11-01,35.48,40.260428,35.48,-4.780428,train,arima_reg,,global
8,2020-12-01,63.98,31.617338,63.98,32.362662,train,arima_reg,,global
9,2021-01-01,109.76,69.716422,109.76,40.043578,train,arima_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,totaltar,-0.301816,0.195180,-1.546350e+00,0.122020,-0.684362,0.080729,NaN,arima_reg,,global
1,mean_med_diesel_crack_input1_trade_month_lag2,-0.060808,0.097095,-6.262692e-01,0.531138,-0.251110,0.129495,NaN,arima_reg,,global
2,mean_nwe_hsfo_crack_trade_month_lag1,-0.160499,0.361724,-4.437072e-01,0.657254,-0.869465,0.548466,NaN,arima_reg,,global
3,ar.L1,0.212695,0.211311,1.006549e+00,0.314151,-0.201467,0.626856,NaN,arima_reg,,global
4,ma.L1,-1.000002,0.259247,-3.857324e+00,0.000115,-1.508117,-0.491886,NaN,arima_reg,,global
5,ar.S.L7,0.011709,0.346587,3.378505e-02,0.973049,-0.667588,0.691007,NaN,arima_reg,,global
6,ma.S.L7,0.314804,0.570340,5.519592e-01,0.580976,-0.803041,1.432650,NaN,arima_reg,,global
7,sigma2,727.308723,0.000356,2.040401e+06,0.000000,727.308024,727.309421,NaN,arima_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,38.152458,train,arima_reg,,global
1,mae,29.125517,train,arima_reg,,global
2,mape,34.139536,train,arima_reg,,global
3,smape,33.933949,train,arima_reg,,global
4,r_squared,-0.171003,train,arima_reg,,global
5,mda,53.658537,train,arima_reg,,global
6,rmse,37.637993,test,arima_reg,,global
7,mae,28.97517,test,arima_reg,,global
8,mape,21.216024,test,arima_reg,,global
9,smape,25.068372,test,arima_reg,,global


## 1.3 - OLS - Statsmodels

In [13]:
from py_parsnip import linear_reg

# Create specification with statsmodels engine
spec_sm = linear_reg().set_engine("statsmodels")

# Fit model
fit_sm = spec_sm.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_sm = fit_sm.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_sm, coefs_sm, stats_sm = fit_sm.extract_outputs()

display(outputs_sm)
display(coefs_sm)
display(stats_sm)

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,103.031150,137.65,34.618850,train,linear_reg,,global
1,2020-05-01,113.53,100.680966,113.53,12.849034,train,linear_reg,,global
2,2020-06-01,43.31,99.272716,43.31,-55.962716,train,linear_reg,,global
3,2020-07-01,79.77,98.337003,79.77,-18.567003,train,linear_reg,,global
4,2020-08-01,48.27,98.790761,48.27,-50.520761,train,linear_reg,,global
5,2020-09-01,98.01,98.953101,98.01,-0.943101,train,linear_reg,,global
6,2020-10-01,49.77,98.915648,49.77,-49.145648,train,linear_reg,,global
7,2020-11-01,35.48,98.973392,35.48,-63.493392,train,linear_reg,,global
8,2020-12-01,63.98,97.883238,63.98,-33.903238,train,linear_reg,,global
9,2021-01-01,109.76,99.092251,109.76,10.667749,train,linear_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,92.420998,8.352429,11.065165,1.903858e-13,75.512390,109.329606,NaN,linear_reg,,global
1,totaltar,-0.274647,0.110660,-2.481899,1.760500e-02,-0.498666,-0.050627,1.004942,linear_reg,,global
2,mean_med_diesel_crack_input1_trade_month_lag2,-0.093034,0.097941,-0.949898,3.481667e-01,-0.291305,0.105237,1.009927,linear_reg,,global
3,mean_nwe_hsfo_crack_trade_month_lag1,-0.470998,0.420807,-1.119275,2.700480e-01,-1.322877,0.380880,1.005225,linear_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,31.772848,train,linear_reg,,global
1,mae,25.035409,train,linear_reg,,global
2,mape,31.042183,train,linear_reg,,global
3,smape,26.008852,train,linear_reg,,global
4,r_squared,0.18787,train,linear_reg,,global
5,mda,58.536585,train,linear_reg,,global
6,adj_r_squared,0.100073,train,linear_reg,,global
7,rmse,39.604179,test,linear_reg,,global
8,mae,31.330025,test,linear_reg,,global
9,mape,23.043006,test,linear_reg,,global


## 1.4 - OLS - sklearn

In [14]:
# sklearn engine (default)
spec_sk = linear_reg()
fit_sk = spec_sk.fit(train_data, FORMULA_STR)
fit_sk = fit_sk.evaluate(test_data)

print("sklearn model fitted successfully!")

outputs_sk, coefs_sk, stats_sk = fit_sk.extract_outputs()
display(outputs_sk)
display(coefs_sk)
display(stats_sk)

sklearn model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,103.031150,137.65,34.618850,train,linear_reg,,global
1,2020-05-01,113.53,100.680966,113.53,12.849034,train,linear_reg,,global
2,2020-06-01,43.31,99.272716,43.31,-55.962716,train,linear_reg,,global
3,2020-07-01,79.77,98.337003,79.77,-18.567003,train,linear_reg,,global
4,2020-08-01,48.27,98.790761,48.27,-50.520761,train,linear_reg,,global
5,2020-09-01,98.01,98.953101,98.01,-0.943101,train,linear_reg,,global
6,2020-10-01,49.77,98.915648,49.77,-49.145648,train,linear_reg,,global
7,2020-11-01,35.48,98.973392,35.48,-63.493392,train,linear_reg,,global
8,2020-12-01,63.98,97.883238,63.98,-33.903238,train,linear_reg,,global
9,2021-01-01,109.76,99.092251,109.76,10.667749,train,linear_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,92.420998,8.352429,11.065165,1.905143e-13,75.512390,109.329606,NaN,linear_reg,,global
1,totaltar,-0.274647,0.110660,-2.481899,1.760500e-02,-0.498666,-0.050627,1.004942,linear_reg,,global
2,mean_med_diesel_crack_input1_trade_month_lag2,-0.093034,0.097941,-0.949898,3.481667e-01,-0.291305,0.105237,1.009927,linear_reg,,global
3,mean_nwe_hsfo_crack_trade_month_lag1,-0.470998,0.420807,-1.119275,2.700480e-01,-1.322877,0.380880,1.005225,linear_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,31.772848,train,linear_reg,,global
1,mae,25.035409,train,linear_reg,,global
2,mape,31.042183,train,linear_reg,,global
3,smape,26.008852,train,linear_reg,,global
4,r_squared,0.18787,train,linear_reg,,global
5,mda,58.536585,train,linear_reg,,global
6,adj_r_squared,0.123755,train,linear_reg,,global
7,rmse,39.604179,test,linear_reg,,global
8,mae,31.330025,test,linear_reg,,global
9,mape,23.043006,test,linear_reg,,global


In [15]:
from py_parsnip import linear_reg

# Create specification with statsmodels engine
spec_elasticnet = linear_reg(penalty = 0.1, mixture = 0.5).set_engine("sklearn")

# Fit model
fit_elasticnet = spec_elasticnet.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_elasticnet = fit_elasticnet.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_elasticnet, coefs_elasticnet, stats_elasticnet = fit_elasticnet.extract_outputs()

display(outputs_elasticnet)
display(coefs_elasticnet)
display(stats_elasticnet)

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,103.030212,137.65,34.619788,train,linear_reg,,global
1,2020-05-01,113.53,100.681260,113.53,12.848740,train,linear_reg,,global
2,2020-06-01,43.31,99.273415,43.31,-55.963415,train,linear_reg,,global
3,2020-07-01,79.77,98.338550,79.77,-18.568550,train,linear_reg,,global
4,2020-08-01,48.27,98.792697,48.27,-50.522697,train,linear_reg,,global
5,2020-09-01,98.01,98.955125,98.01,-0.945125,train,linear_reg,,global
6,2020-10-01,49.77,98.917902,49.77,-49.147902,train,linear_reg,,global
7,2020-11-01,35.48,98.975496,35.48,-63.495496,train,linear_reg,,global
8,2020-12-01,63.98,97.886346,63.98,-33.906346,train,linear_reg,,global
9,2021-01-01,109.76,99.094081,109.76,10.665919,train,linear_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,92.426157,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
1,totaltar,-0.274617,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
2,mean_med_diesel_crack_input1_trade_month_lag2,-0.093024,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
3,mean_nwe_hsfo_crack_trade_month_lag1,-0.470513,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,31.772848,train,linear_reg,,global
1,mae,25.035688,train,linear_reg,,global
2,mape,31.043648,train,linear_reg,,global
3,smape,26.010013,train,linear_reg,,global
4,r_squared,0.18787,train,linear_reg,,global
5,mda,58.536585,train,linear_reg,,global
6,adj_r_squared,0.123755,train,linear_reg,,global
7,rmse,39.601142,test,linear_reg,,global
8,mae,31.327756,test,linear_reg,,global
9,mape,23.041318,test,linear_reg,,global


In [16]:
from py_parsnip import linear_reg

# Create specification with statsmodels engine
spec_ridge  = linear_reg(penalty = 0.1, mixture = 0).set_engine("sklearn")

# Fit model
fit_ridge = spec_ridge.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_ridge = fit_ridge.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_ridge, coefs_ridge, stats_ridge = fit_ridge.extract_outputs()

display(outputs_ridge)
display(coefs_ridge)
display(stats_ridge)

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,103.031138,137.65,34.618862,train,linear_reg,,global
1,2020-05-01,113.53,100.680970,113.53,12.849030,train,linear_reg,,global
2,2020-06-01,43.31,99.272724,43.31,-55.962724,train,linear_reg,,global
3,2020-07-01,79.77,98.337024,79.77,-18.567024,train,linear_reg,,global
4,2020-08-01,48.27,98.790790,48.27,-50.520790,train,linear_reg,,global
5,2020-09-01,98.01,98.953132,98.01,-0.943132,train,linear_reg,,global
6,2020-10-01,49.77,98.915682,49.77,-49.145682,train,linear_reg,,global
7,2020-11-01,35.48,98.973424,35.48,-63.493424,train,linear_reg,,global
8,2020-12-01,63.98,97.883285,63.98,-33.903285,train,linear_reg,,global
9,2021-01-01,109.76,99.092279,109.76,10.667721,train,linear_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,92.421068,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
1,totaltar,-0.274646,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
2,mean_med_diesel_crack_input1_trade_month_lag2,-0.093034,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
3,mean_nwe_hsfo_crack_trade_month_lag1,-0.470991,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,31.772848,train,linear_reg,,global
1,mae,25.035414,train,linear_reg,,global
2,mape,31.042204,train,linear_reg,,global
3,smape,26.008867,train,linear_reg,,global
4,r_squared,0.18787,train,linear_reg,,global
5,mda,58.536585,train,linear_reg,,global
6,adj_r_squared,0.123755,train,linear_reg,,global
7,rmse,39.604131,test,linear_reg,,global
8,mae,31.329987,test,linear_reg,,global
9,mape,23.042977,test,linear_reg,,global


In [17]:
from py_parsnip import linear_reg

# Create specification with statsmodels engine
spec_lasso  = linear_reg(penalty = 0.1, mixture = 1).set_engine("sklearn")

# Fit model
fit_lasso = spec_lasso.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_lasso = fit_lasso.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_lasso, coefs_lasso, stats_lasso = fit_lasso.extract_outputs()

display(outputs_lasso)
display(coefs_lasso)
display(stats_lasso)

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,103.029775,137.65,34.620225,train,linear_reg,,global
1,2020-05-01,113.53,100.681384,113.53,12.848616,train,linear_reg,,global
2,2020-06-01,43.31,99.273792,43.31,-55.963792,train,linear_reg,,global
3,2020-07-01,79.77,98.339240,79.77,-18.569240,train,linear_reg,,global
4,2020-08-01,48.27,98.793435,48.27,-50.523435,train,linear_reg,,global
5,2020-09-01,98.01,98.955868,98.01,-0.945868,train,linear_reg,,global
6,2020-10-01,49.77,98.918707,49.77,-49.148707,train,linear_reg,,global
7,2020-11-01,35.48,98.976257,35.48,-63.496257,train,linear_reg,,global
8,2020-12-01,63.98,97.887476,63.98,-33.907476,train,linear_reg,,global
9,2021-01-01,109.76,99.094761,109.76,10.665239,train,linear_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,92.428387,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
1,totaltar,-0.274601,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
2,mean_med_diesel_crack_input1_trade_month_lag2,-0.093012,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
3,mean_nwe_hsfo_crack_trade_month_lag1,-0.470341,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,31.772849,train,linear_reg,,global
1,mae,25.035772,train,linear_reg,,global
2,mape,31.044268,train,linear_reg,,global
3,smape,26.010527,train,linear_reg,,global
4,r_squared,0.18787,train,linear_reg,,global
5,mda,58.536585,train,linear_reg,,global
6,adj_r_squared,0.123755,train,linear_reg,,global
7,rmse,39.60012,test,linear_reg,,global
8,mae,31.327088,test,linear_reg,,global
9,mape,23.040836,test,linear_reg,,global


In [18]:
from py_parsnip import gen_additive_mod

# Create specification with statsmodels engine
spec_gam  = gen_additive_mod()

# Fit model
fit_gam = spec_gam.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_gam = fit_gam.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_gam, coefs_gam, stats_gam = fit_gam.extract_outputs()
display(outputs_gam)
display(coefs_gam)
display(stats_gam)

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,100.416486,137.65,37.233514,train,gen_additive_mod,,global
1,2020-05-01,113.53,88.870157,113.53,24.659843,train,gen_additive_mod,,global
2,2020-06-01,43.31,81.534833,43.31,-38.224833,train,gen_additive_mod,,global
3,2020-07-01,79.77,78.245820,79.77,1.524180,train,gen_additive_mod,,global
4,2020-08-01,48.27,81.788009,48.27,-33.518009,train,gen_additive_mod,,global
5,2020-09-01,98.01,83.113151,98.01,14.896849,train,gen_additive_mod,,global
6,2020-10-01,49.77,83.700077,49.77,-33.930077,train,gen_additive_mod,,global
7,2020-11-01,35.48,83.529688,35.48,-48.049688,train,gen_additive_mod,,global
8,2020-12-01,63.98,80.368959,63.98,-16.388959,train,gen_additive_mod,,global
9,2021-01-01,109.76,83.355294,109.76,26.404706,train,gen_additive_mod,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,gen_additive_mod,,global
1,totaltar,95.202059,NaN,NaN,NaN,NaN,NaN,NaN,gen_additive_mod,,global
2,mean_med_diesel_crack_input1_trade_month_lag2,26.058629,NaN,NaN,NaN,NaN,NaN,NaN,gen_additive_mod,,global
3,mean_nwe_hsfo_crack_trade_month_lag1,36.878530,NaN,NaN,NaN,NaN,NaN,NaN,gen_additive_mod,,global


,metric,value,split,model,model_group_name,group
0,rmse,27.437815,train,gen_additive_mod,,global
1,mae,23.192792,train,gen_additive_mod,,global
2,mape,27.842869,train,gen_additive_mod,,global
3,smape,24.527053,train,gen_additive_mod,,global
4,r_squared,0.394363,train,gen_additive_mod,,global
5,mda,65.853659,train,gen_additive_mod,,global
6,adj_r_squared,0.219982,train,gen_additive_mod,,global
7,rmse,41.489347,test,gen_additive_mod,,global
8,mae,33.758892,test,gen_additive_mod,,global
9,mape,25.374214,test,gen_additive_mod,,global


In [19]:
# from py_parsnip import arima_reg

# # Create specification with statsmodels engine
# spec_auto_arima  = arima_reg().set_engine("auto_arima")

# # Fit model
# fit_auto_arima = spec_auto_arima.fit(train_data, FORMULA_STR)

# # Evaluate on test set
# fit_auto_arima = fit_auto_arima.evaluate(test_data)

# print("Model fitted successfully!")

# # Extract outputs from both models
# outputs_auto_arima, coefs_auto_arima, stats_auto_arima = fit_auto_arima.extract_outputs()
# display(outputs_auto_arima)
# display(coefs_auto_arima)
# display(stats_auto_arima)

In [20]:
from py_parsnip import prophet_reg

# Create specification with statsmodels engine
spec_prophet  = prophet_reg()

# Fit model
fit_prophet = spec_prophet.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_prophet = fit_prophet.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_prophet, coefs_prophet, stats_prophet = fit_prophet.extract_outputs()
display(outputs_prophet)
display(coefs_prophet)
display(stats_prophet)

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,112.812329,137.65,24.837671,train,prophet_reg,,global
1,2020-05-01,113.53,115.589899,113.53,-2.059899,train,prophet_reg,,global
2,2020-06-01,43.31,66.349488,43.31,-23.039488,train,prophet_reg,,global
3,2020-07-01,79.77,70.154397,79.77,9.615603,train,prophet_reg,,global
4,2020-08-01,48.27,64.104879,48.27,-15.834879,train,prophet_reg,,global
5,2020-09-01,98.01,118.531915,98.01,-20.521915,train,prophet_reg,,global
6,2020-10-01,49.77,79.545138,49.77,-29.775138,train,prophet_reg,,global
7,2020-11-01,35.48,58.856986,35.48,-23.376986,train,prophet_reg,,global
8,2020-12-01,63.98,99.834320,63.98,-35.854320,train,prophet_reg,,global
9,2021-01-01,109.76,115.407468,109.76,-5.647468,train,prophet_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,growth,linear,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
1,changepoint_prior_scale,0.05,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
2,seasonality_prior_scale,10.0,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
3,seasonality_mode,additive,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
4,n_changepoints,25,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,21.537087,train,prophet_reg,,global
1,mae,17.262224,train,prophet_reg,,global
2,mape,19.910901,train,prophet_reg,,global
3,smape,18.557042,train,prophet_reg,,global
4,r_squared,0.626847,train,prophet_reg,,global
5,mda,80.487805,train,prophet_reg,,global
6,rmse,45.21693,test,prophet_reg,,global
7,mae,38.048444,test,prophet_reg,,global
8,mape,29.854629,test,prophet_reg,,global
9,smape,32.221975,test,prophet_reg,,global


In [21]:
from py_parsnip import exp_smoothing

# Create specification with statsmodels engine
spec_exp_smoothing  = exp_smoothing(error="add", trend="add", season="add", seasonal_period=12)

# Fit model
fit_exp_smoothing = spec_exp_smoothing.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_exp_smoothing = fit_exp_smoothing.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_exp_smoothing, coefs_exp_smoothing, stats_exp_smoothing = fit_exp_smoothing.extract_outputs()
display(outputs_exp_smoothing)
display(coefs_exp_smoothing)
display(stats_exp_smoothing)

Model fitted successfully!


/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.



,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,119.568266,137.65,18.081734,train,exp_smoothing,,global
1,2020-05-01,113.53,102.034604,113.53,11.495396,train,exp_smoothing,,global
2,2020-06-01,43.31,72.949596,43.31,-29.639596,train,exp_smoothing,,global
3,2020-07-01,79.77,90.679617,79.77,-10.909617,train,exp_smoothing,,global
4,2020-08-01,48.27,78.521668,48.27,-30.251668,train,exp_smoothing,,global
5,2020-09-01,98.01,69.405629,98.01,28.604371,train,exp_smoothing,,global
6,2020-10-01,49.77,61.577041,49.77,-11.807041,train,exp_smoothing,,global
7,2020-11-01,35.48,89.866576,35.48,-54.386576,train,exp_smoothing,,global
8,2020-12-01,63.98,112.668013,63.98,-48.688013,train,exp_smoothing,,global
9,2021-01-01,109.76,94.222961,109.76,15.537039,train,exp_smoothing,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,alpha (smoothing_level),1.490116e-08,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
1,beta (smoothing_trend),0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
2,gamma (smoothing_seasonal),0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
3,phi (damping_trend),NaN,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
4,l0 (initial_level),8.749568e+01,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
5,b0 (initial_trend),4.893895e-01,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global


,metric,value,split,model,model_group_name,group
0,rmse,29.174087,train,exp_smoothing,,global
1,mae,24.100522,train,exp_smoothing,,global
2,mape,32.149773,train,exp_smoothing,,global
3,smape,26.919124,train,exp_smoothing,,global
4,r_squared,0.315289,train,exp_smoothing,,global
5,mda,68.292683,train,exp_smoothing,,global
6,rmse,26.522722,test,exp_smoothing,,global
7,mae,20.040026,test,exp_smoothing,,global
8,mape,15.517037,test,exp_smoothing,,global
9,smape,17.038668,test,exp_smoothing,,global


In [22]:
from py_parsnip import arima_boost

# Create specification with statsmodels engine
spec_arima_boost  = arima_boost()

# Fit model
fit_arima_boost = spec_arima_boost.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_arima_boost = fit_arima_boost.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_arima_boost, coefs_arima_boost, stats_arima_boost = fit_arima_boost.extract_outputs()
display(outputs_arima_boost)
display(coefs_arima_boost)
display(stats_arima_boost)

Model fitted successfully!


/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.



,date,actuals,arima_fitted,xgb_fitted,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,69.644426,66.296181,135.940607,137.65,1.709393,train,arima_boost,,global
1,2020-05-01,113.53,53.958858,60.213924,114.172782,113.53,-0.642782,train,arima_boost,,global
2,2020-06-01,43.31,43.831392,1.181154,45.012545,43.31,-1.702545,train,arima_boost,,global
3,2020-07-01,79.77,38.368178,41.388634,79.756812,79.77,0.013188,train,arima_boost,,global
4,2020-08-01,48.27,42.765946,5.786700,48.552646,48.27,-0.282646,train,arima_boost,,global
5,2020-09-01,98.01,44.227059,53.478550,97.705609,98.01,0.304391,train,arima_boost,,global
6,2020-10-01,49.77,44.438479,5.319664,49.758143,49.77,0.011857,train,arima_boost,,global
7,2020-11-01,35.48,44.562212,-7.320201,37.242012,35.48,-1.762012,train,arima_boost,,global
8,2020-12-01,63.98,38.231713,25.720531,63.952244,63.98,0.027756,train,arima_boost,,global
9,2021-01-01,109.76,44.892472,63.695606,108.588078,109.76,1.171922,train,arima_boost,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,arima_totaltar,-0.047637,0.137863,-0.345539,7.296893e-01,NaN,NaN,NaN,arima_boost,,global
1,arima_mean_med_diesel_crack_input1_trade_month...,-0.706213,0.134381,-5.255296,1.477866e-07,NaN,NaN,NaN,arima_boost,,global
2,arima_mean_nwe_hsfo_crack_trade_month_lag1,-2.621992,0.557571,-4.702529,2.569593e-06,NaN,NaN,NaN,arima_boost,,global
3,arima_sigma2,4255.375062,1046.004270,4.068220,4.737373e-05,NaN,NaN,NaN,arima_boost,,global
4,xgb_n_estimators,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,arima_boost,,global
5,xgb_max_depth,6.000000,NaN,NaN,NaN,NaN,NaN,NaN,arima_boost,,global
6,xgb_learning_rate,0.100000,NaN,NaN,NaN,NaN,NaN,NaN,arima_boost,,global
7,xgb_min_child_weight,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,arima_boost,,global
8,xgb_gamma,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,arima_boost,,global
9,xgb_subsample,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,arima_boost,,global


,metric,value,split,model,model_group_name,group
0,rmse,1.043832,train,arima_boost,,global
1,mae,0.756105,train,arima_boost,,global
2,mape,0.939863,train,arima_boost,,global
3,smape,0.931641,train,arima_boost,,global
4,r_squared,0.999123,train,arima_boost,,global
5,mda,97.560976,train,arima_boost,,global
6,rmse,52.801972,test,arima_boost,,global
7,mae,40.741473,test,arima_boost,,global
8,mape,30.73324,test,arima_boost,,global
9,smape,39.25544,test,arima_boost,,global


In [23]:
from py_parsnip import prophet_boost

# Create specification with statsmodels engine
spec_prophet_boost  = prophet_boost(     
      seasonality_yearly=False,
      seasonality_weekly=False,
      seasonality_daily=False)   

# Fit model
fit_prophet_boost = spec_prophet_boost.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_prophet_boost = fit_prophet_boost.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_prophet_boost, coefs_prophet_boost, stats_prophet_boost = fit_prophet_boost.extract_outputs()
display(outputs_prophet_boost)
display(coefs_prophet_boost)
display(stats_prophet_boost)

Model fitted successfully!


,date,actuals,prophet_fitted,xgb_fitted,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,112.553041,24.644632,137.197673,137.65,0.452327,train,prophet_boost,,global
1,2020-05-01,113.53,122.000742,-8.389091,113.611651,113.53,-0.081651,train,prophet_boost,,global
2,2020-06-01,43.31,70.450176,-26.951851,43.498325,43.31,-0.188325,train,prophet_boost,,global
3,2020-07-01,79.77,71.641046,7.949687,79.590733,79.77,0.179267,train,prophet_boost,,global
4,2020-08-01,48.27,59.675524,-11.255198,48.420327,48.27,-0.150327,train,prophet_boost,,global
5,2020-09-01,98.01,109.218005,-11.491523,97.726482,98.01,0.283518,train,prophet_boost,,global
6,2020-10-01,49.77,75.128970,-25.275051,49.853919,49.77,-0.083919,train,prophet_boost,,global
7,2020-11-01,35.48,55.915862,-20.377525,35.538337,35.48,-0.058337,train,prophet_boost,,global
8,2020-12-01,63.98,96.949583,-32.496399,64.453184,63.98,-0.473184,train,prophet_boost,,global
9,2021-01-01,109.76,116.987087,-7.293860,109.693227,109.76,0.066773,train,prophet_boost,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,prophet_growth,linear,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
1,prophet_changepoint_prior_scale,0.05,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
2,prophet_seasonality_prior_scale,10.0,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
3,prophet_seasonality_mode,additive,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
4,prophet_n_changepoints,25,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
5,prophet_changepoint_range,0.8,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
6,prophet_n_changepoints_detected,25,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
7,xgb_n_estimators,100,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
8,xgb_max_depth,6,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
9,xgb_learning_rate,0.1,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global


,metric,value,split,model,model_group_name,group
0,rmse,0.242193,train,prophet_boost,,global
1,mae,0.189822,train,prophet_boost,,global
2,mape,0.21857,train,prophet_boost,,global
3,smape,0.21829,train,prophet_boost,,global
4,r_squared,0.999953,train,prophet_boost,,global
5,mda,97.560976,train,prophet_boost,,global
6,rmse,39.859288,test,prophet_boost,,global
7,mae,32.367424,test,prophet_boost,,global
8,mape,25.33341,test,prophet_boost,,global
9,smape,28.699086,test,prophet_boost,,global


In [24]:
from py_parsnip import boost_tree   

# Create specification with statsmodels engine
spec_boost_tree  = boost_tree().set_engine("lightgbm")

# Fit model
fit_boost_tree = spec_boost_tree.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_boost_tree = fit_boost_tree.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_boost_tree, coefs_boost_tree, stats_boost_tree = fit_boost_tree.extract_outputs()
display(outputs_boost_tree)
display(coefs_boost_tree)
display(stats_boost_tree)

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,96.932889,137.65,40.717111,train,boost_tree,,global
1,2020-05-01,113.53,88.129225,113.53,25.400775,train,boost_tree,,global
2,2020-06-01,43.31,88.129225,43.31,-44.819225,train,boost_tree,,global
3,2020-07-01,79.77,88.129225,79.77,-8.359225,train,boost_tree,,global
4,2020-08-01,48.27,88.129225,48.27,-39.859225,train,boost_tree,,global
5,2020-09-01,98.01,88.129225,98.01,9.880775,train,boost_tree,,global
6,2020-10-01,49.77,88.129225,49.77,-38.359225,train,boost_tree,,global
7,2020-11-01,35.48,88.129225,35.48,-52.649225,train,boost_tree,,global
8,2020-12-01,63.98,88.129225,63.98,-24.149225,train,boost_tree,,global
9,2021-01-01,109.76,88.129225,109.76,21.630775,train,boost_tree,,global


,variable,importance,model,model_group_name,group
0,mean_med_diesel_crack_input1_trade_month_lag2,52,boost_tree,,global
1,mean_nwe_hsfo_crack_trade_month_lag1,48,boost_tree,,global
2,Intercept,0,boost_tree,,global
3,totaltar,0,boost_tree,,global


,metric,value,split,model,model_group_name,group
0,rmse,34.09723,train,boost_tree,,global
1,mae,27.775788,train,boost_tree,,global
2,mape,38.451064,train,boost_tree,,global
3,smape,30.102904,train,boost_tree,,global
4,r_squared,0.064699,train,boost_tree,,global
5,mda,51.219512,train,boost_tree,,global
6,rmse,28.19205,test,boost_tree,,global
7,mae,21.750719,test,boost_tree,,global
8,mape,15.849579,test,boost_tree,,global
9,smape,17.886132,test,boost_tree,,global


In [25]:
from py_parsnip import null_model

# Create specification with statsmodels engine
spec_null_model  = null_model(strategy="mean")

# Fit model
fit_null_model = spec_null_model.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_null_model = fit_null_model.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_null_model, coefs_null_model, stats_null_model = fit_null_model.extract_outputs()
display(outputs_null_model)
display(coefs_null_model)
display(stats_null_model)

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,99.005,99.005,38.645,train,null_model,,global
1,2020-05-01,113.53,99.005,99.005,14.525,train,null_model,,global
2,2020-06-01,43.31,99.005,99.005,-55.695,train,null_model,,global
3,2020-07-01,79.77,99.005,99.005,-19.235,train,null_model,,global
4,2020-08-01,48.27,99.005,99.005,-50.735,train,null_model,,global
5,2020-09-01,98.01,99.005,99.005,-0.995,train,null_model,,global
6,2020-10-01,49.77,99.005,99.005,-49.235,train,null_model,,global
7,2020-11-01,35.48,99.005,99.005,-63.525,train,null_model,,global
8,2020-12-01,63.98,99.005,99.005,-35.025,train,null_model,,global
9,2021-01-01,109.76,99.005,99.005,10.755,train,null_model,,global


,variable,coefficient,std_error,p_value,ci_0.025,ci_0.975,model,model_group_name,group
0,(Intercept),99.005,NaN,NaN,NaN,NaN,null_model,,global


,metric,value,split,model,model_group_name,group
0,rmse,35.256848,train,null_model,,global
1,mae,27.009762,train,null_model,,global
2,mape,38.484528,train,null_model,,global
3,r_squared,0.0,train,null_model,,global
4,baseline_value,99.005,train,null_model,,global
5,rmse,32.591708,test,null_model,,global
6,mae,25.992333,test,null_model,,global
7,mape,19.028925,test,null_model,,global
8,r_squared,-1.747465,test,null_model,,global
9,train_start_date,2020-04-01T00:00:00.000000000,train,null_model,,global


In [26]:
from py_parsnip import naive_reg

# Create specification with statsmodels engine
spec_naive_model  = naive_reg(strategy="naive")

# Fit model
fit_naive_model = spec_naive_model.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_naive_model = fit_naive_model.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_naive_model, coefs_naive_model, stats_naive_model = fit_naive_model.extract_outputs()
display(outputs_naive_model)
display(coefs_naive_model)
display(stats_naive_model)

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,137.65,137.65,0.00,train,naive_reg,,global
1,2020-05-01,113.53,137.65,137.65,-24.12,train,naive_reg,,global
2,2020-06-01,43.31,113.53,113.53,-70.22,train,naive_reg,,global
3,2020-07-01,79.77,43.31,43.31,36.46,train,naive_reg,,global
4,2020-08-01,48.27,79.77,79.77,-31.50,train,naive_reg,,global
5,2020-09-01,98.01,48.27,48.27,49.74,train,naive_reg,,global
6,2020-10-01,49.77,98.01,98.01,-48.24,train,naive_reg,,global
7,2020-11-01,35.48,49.77,49.77,-14.29,train,naive_reg,,global
8,2020-12-01,63.98,35.48,35.48,28.50,train,naive_reg,,global
9,2021-01-01,109.76,63.98,63.98,45.78,train,naive_reg,,global


,variable,coefficient,std_error,p_value,ci_0.025,ci_0.975,model,model_group_name,group
0,strategy,naive,NaN,NaN,NaN,NaN,naive_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,39.867462,train,naive_reg,,global
1,mae,30.887619,train,naive_reg,,global
2,mape,38.600982,train,naive_reg,,global
3,r_squared,-0.278646,train,naive_reg,,global
4,strategy,naive,train,naive_reg,,global
5,rmse,69.468119,test,naive_reg,,global
6,mae,66.627333,test,naive_reg,,global
7,mape,52.262193,test,naive_reg,,global
8,r_squared,-11.482139,test,naive_reg,,global
9,train_start_date,2020-04-01T00:00:00.000000000,train,naive_reg,,global


In [27]:
from py_parsnip import naive_reg

# Create specification with statsmodels engine
spec_naive_model  = naive_reg(strategy="seasonal_naive", seasonal_period=12)

# Fit model
fit_naive_model = spec_naive_model.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_naive_model = fit_naive_model.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_naive_model, coefs_naive_model, stats_naive_model = fit_naive_model.extract_outputs()
display(outputs_naive_model)
display(coefs_naive_model)
display(stats_naive_model)

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,137.65,137.65,0.00,train,naive_reg,,global
1,2020-05-01,113.53,113.53,113.53,0.00,train,naive_reg,,global
2,2020-06-01,43.31,43.31,43.31,0.00,train,naive_reg,,global
3,2020-07-01,79.77,79.77,79.77,0.00,train,naive_reg,,global
4,2020-08-01,48.27,48.27,48.27,0.00,train,naive_reg,,global
5,2020-09-01,98.01,98.01,98.01,0.00,train,naive_reg,,global
6,2020-10-01,49.77,49.77,49.77,0.00,train,naive_reg,,global
7,2020-11-01,35.48,35.48,35.48,0.00,train,naive_reg,,global
8,2020-12-01,63.98,63.98,63.98,0.00,train,naive_reg,,global
9,2021-01-01,109.76,109.76,109.76,0.00,train,naive_reg,,global


,variable,coefficient,std_error,p_value,ci_0.025,ci_0.975,model,model_group_name,group
0,strategy,seasonal_naive,NaN,NaN,NaN,NaN,naive_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,42.541986,train,naive_reg,,global
1,mae,27.645714,train,naive_reg,,global
2,mape,33.286186,train,naive_reg,,global
3,r_squared,-0.455957,train,naive_reg,,global
4,strategy,seasonal_naive,train,naive_reg,,global
5,rmse,44.570802,test,naive_reg,,global
6,mae,31.266,test,naive_reg,,global
7,mape,24.715076,test,naive_reg,,global
8,r_squared,-4.138292,test,naive_reg,,global
9,train_start_date,2020-04-01T00:00:00.000000000,train,naive_reg,,global


In [28]:
from py_parsnip import naive_reg

# Create specification with statsmodels engine
spec_naive_model  = naive_reg(strategy="drift")

# Fit model
fit_naive_model = spec_naive_model.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_naive_model = fit_naive_model.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_naive_model, coefs_naive_model, stats_naive_model = fit_naive_model.extract_outputs()
display(outputs_naive_model)
display(coefs_naive_model)
display(stats_naive_model)

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,137.650000,137.650000,0.000000,train,naive_reg,,global
1,2020-05-01,113.53,135.716341,135.716341,-22.186341,train,naive_reg,,global
2,2020-06-01,43.31,111.596341,111.596341,-68.286341,train,naive_reg,,global
3,2020-07-01,79.77,41.376341,41.376341,38.393659,train,naive_reg,,global
4,2020-08-01,48.27,77.836341,77.836341,-29.566341,train,naive_reg,,global
5,2020-09-01,98.01,46.336341,46.336341,51.673659,train,naive_reg,,global
6,2020-10-01,49.77,96.076341,96.076341,-46.306341,train,naive_reg,,global
7,2020-11-01,35.48,47.836341,47.836341,-12.356341,train,naive_reg,,global
8,2020-12-01,63.98,33.546341,33.546341,30.433659,train,naive_reg,,global
9,2021-01-01,109.76,62.046341,62.046341,47.713659,train,naive_reg,,global


,variable,coefficient,std_error,p_value,ci_0.025,ci_0.975,model,model_group_name,group
0,strategy,drift,NaN,NaN,NaN,NaN,naive_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,39.821659,train,naive_reg,,global
1,mae,30.808722,train,naive_reg,,global
2,mape,38.155328,train,naive_reg,,global
3,r_squared,-0.275709,train,naive_reg,,global
4,strategy,drift,train,naive_reg,,global
5,rmse,84.801318,test,naive_reg,,global
6,mae,82.096602,test,naive_reg,,global
7,mape,64.940391,test,naive_reg,,global
8,r_squared,-17.600436,test,naive_reg,,global
9,train_start_date,2020-04-01T00:00:00.000000000,train,naive_reg,,global


In [29]:
from py_parsnip import null_model

# Create specification with statsmodels engine
spec_null_model  = null_model(strategy="mean")

# Fit model
fit_null_model = spec_null_model.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_null_model = fit_null_model.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_null_model, coefs_null_model, stats_null_model = fit_null_model.extract_outputs()
display(outputs_null_model)
display(coefs_null_model)
display(stats_null_model)

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,99.005,99.005,38.645,train,null_model,,global
1,2020-05-01,113.53,99.005,99.005,14.525,train,null_model,,global
2,2020-06-01,43.31,99.005,99.005,-55.695,train,null_model,,global
3,2020-07-01,79.77,99.005,99.005,-19.235,train,null_model,,global
4,2020-08-01,48.27,99.005,99.005,-50.735,train,null_model,,global
5,2020-09-01,98.01,99.005,99.005,-0.995,train,null_model,,global
6,2020-10-01,49.77,99.005,99.005,-49.235,train,null_model,,global
7,2020-11-01,35.48,99.005,99.005,-63.525,train,null_model,,global
8,2020-12-01,63.98,99.005,99.005,-35.025,train,null_model,,global
9,2021-01-01,109.76,99.005,99.005,10.755,train,null_model,,global


,variable,coefficient,std_error,p_value,ci_0.025,ci_0.975,model,model_group_name,group
0,(Intercept),99.005,NaN,NaN,NaN,NaN,null_model,,global


,metric,value,split,model,model_group_name,group
0,rmse,35.256848,train,null_model,,global
1,mae,27.009762,train,null_model,,global
2,mape,38.484528,train,null_model,,global
3,r_squared,0.0,train,null_model,,global
4,baseline_value,99.005,train,null_model,,global
5,rmse,32.591708,test,null_model,,global
6,mae,25.992333,test,null_model,,global
7,mape,19.028925,test,null_model,,global
8,r_squared,-1.747465,test,null_model,,global
9,train_start_date,2020-04-01T00:00:00.000000000,train,null_model,,global


## Hybrid Model

Combine two models with different strategies (residual, sequential, weighted)

In [30]:
from py_parsnip import hybrid_model, linear_reg, rand_forest

# Create specification - combine linear regression with random forest
# Strategy: 'residual' trains model2 on residuals from model1
spec_hybrid_model = hybrid_model(
    model1=linear_reg(),
    model2=rand_forest().set_mode('regression'),
    strategy='residual'
)

# Fit model
fit_hybrid_model = spec_hybrid_model.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_hybrid_model = fit_hybrid_model.evaluate(test_data)

print("Hybrid model fitted successfully!")

# Extract outputs from hybrid model
outputs_hybrid_model, coefs_hybrid_model, stats_hybrid_model = fit_hybrid_model.extract_outputs()
display(outputs_hybrid_model)
display(coefs_hybrid_model)
display(stats_hybrid_model)

Hybrid model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,121.965105,137.65,15.684895,train,hybrid_model,,global
1,2020-05-01,113.53,107.662008,113.53,5.867992,train,hybrid_model,,global
2,2020-06-01,43.31,64.574786,43.31,-21.264786,train,hybrid_model,,global
3,2020-07-01,79.77,81.405328,79.77,-1.635328,train,hybrid_model,,global
4,2020-08-01,48.27,68.747004,48.27,-20.477004,train,hybrid_model,,global
5,2020-09-01,98.01,79.830918,98.01,18.179082,train,hybrid_model,,global
6,2020-10-01,49.77,51.930795,49.77,-2.160795,train,hybrid_model,,global
7,2020-11-01,35.48,49.438923,35.48,-13.958923,train,hybrid_model,,global
8,2020-12-01,63.98,64.076147,63.98,-0.096147,train,hybrid_model,,global
9,2021-01-01,109.76,100.198090,109.76,9.561910,train,hybrid_model,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,strategy,residual,NaN,NaN,NaN,NaN,NaN,NaN,hybrid_model,,global
1,model1_type,linear_reg,NaN,NaN,NaN,NaN,NaN,NaN,hybrid_model,,global
2,model2_type,rand_forest,NaN,NaN,NaN,NaN,NaN,NaN,hybrid_model,,global


,metric,value,split,model,model_group_name,group
0,rmse,12.477728,train,hybrid_model,,global
1,mae,9.824556,train,hybrid_model,,global
2,mape,11.77622,train,hybrid_model,,global
3,r_squared,0.874748,train,hybrid_model,,global
4,rmse,41.892818,test,hybrid_model,,global
5,mae,32.902757,test,hybrid_model,,global
6,mape,24.486087,test,hybrid_model,,global
7,r_squared,-3.539386,test,hybrid_model,,global
8,formula,target ~ totaltar +mean_med_diesel_crack_input...,,hybrid_model,,global
9,model_type,hybrid_model,,hybrid_model,,global


## Manual Regression

Specify coefficients manually for comparison with external forecasts or domain knowledge

In [31]:
from py_parsnip import manual_reg

# Create specification with manually specified coefficients
# Useful for comparing with external models or incorporating domain knowledge
spec_manual_reg = manual_reg(
    coefficients={"totaltar": 0.8, "mean_med_diesel_crack_input1_trade_month_lag2": 0.1, "mean_nwe_hsfo_crack_trade_month_lag1": 2},  # Example coefficients
    intercept=100.0
)

# Fit model (validates coefficients and calculates fitted values)
fit_manual_reg = spec_manual_reg.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_manual_reg = fit_manual_reg.evaluate(test_data)

print("Manual regression model fitted successfully!")

# Extract outputs from manual model
outputs_manual_reg, coefs_manual_reg, stats_manual_reg = fit_manual_reg.extract_outputs()
display(outputs_manual_reg)
display(coefs_manual_reg)
display(stats_manual_reg)

Manual regression model fitted successfully!


,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,137.65,71.522,137.65,66.128,train,manual_reg,,global
1,113.53,77.424,113.53,36.106,train,manual_reg,,global
2,43.31,79.819,43.31,-36.509,train,manual_reg,,global
3,79.77,83.394,79.77,-3.624,train,manual_reg,,global
4,48.27,84.400,48.27,-36.130,train,manual_reg,,global
5,98.01,84.584,98.01,13.426,train,manual_reg,,global
6,49.77,85.401,49.77,-35.631,train,manual_reg,,global
7,35.48,84.846,35.48,-49.366,train,manual_reg,,global
8,63.98,89.065,63.98,-25.085,train,manual_reg,,global
9,109.76,83.822,109.76,25.938,train,manual_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,100.0,NaN,NaN,NaN,NaN,NaN,NaN,manual_reg,,global
1,totaltar,0.8,NaN,NaN,NaN,NaN,NaN,NaN,manual_reg,,global
2,mean_med_diesel_crack_input1_trade_month_lag2,0.1,NaN,NaN,NaN,NaN,NaN,NaN,manual_reg,,global
3,mean_nwe_hsfo_crack_trade_month_lag1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,manual_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,69.933254,train,manual_reg,,global
1,mae,49.314381,train,manual_reg,,global
2,mape,72.450355,train,manual_reg,,global
3,r_squared,-2.934418,train,manual_reg,,global
4,rmse,32.714562,test,manual_reg,,global
5,mae,25.8094,test,manual_reg,,global
6,mape,20.741383,test,manual_reg,,global
7,r_squared,-1.768217,test,manual_reg,,global
8,formula,target ~ totaltar +mean_med_diesel_crack_input...,,manual_reg,,global
9,model_type,manual_reg,,manual_reg,,global


In [32]:
spec_boost_xgb = boost_tree().set_engine('catboost')

fit_boost = spec_boost_xgb.fit(train_data,FORMULA_STR,original_training_data=train_data)
fit_boost = fit_boost.evaluate(test_data,original_test_data=test_data)

'fit_boost = spec_boost_xgb.fit(train_data,FORMULA_STR)'
'fit_boost = fit_boost.evaluate(test_data)'

outputs_boost, coefs_boost, stats_boost = fit_boost.extract_outputs()

display(outputs_boost)
display(coefs_boost)
display(stats_boost)

,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,133.853924,137.65,3.796076,train,boost_tree,,global
1,2020-05-01,113.53,114.106366,113.53,-0.576366,train,boost_tree,,global
2,2020-06-01,43.31,47.679499,43.31,-4.369499,train,boost_tree,,global
3,2020-07-01,79.77,79.962675,79.77,-0.192675,train,boost_tree,,global
4,2020-08-01,48.27,49.670994,48.27,-1.400994,train,boost_tree,,global
5,2020-09-01,98.01,96.263157,98.01,1.746843,train,boost_tree,,global
6,2020-10-01,49.77,48.618400,49.77,1.151600,train,boost_tree,,global
7,2020-11-01,35.48,39.331033,35.48,-3.851033,train,boost_tree,,global
8,2020-12-01,63.98,64.060276,63.98,-0.080276,train,boost_tree,,global
9,2021-01-01,109.76,107.384896,109.76,2.375104,train,boost_tree,,global


,variable,importance,model,model_group_name,group
0,mean_nwe_hsfo_crack_trade_month_lag1,50.227384,boost_tree,,global
1,mean_med_diesel_crack_input1_trade_month_lag2,41.694037,boost_tree,,global
2,totaltar,8.078579,boost_tree,,global
3,Intercept,0.000000,boost_tree,,global


,metric,value,split,model,model_group_name,group
0,rmse,2.333801,train,boost_tree,,global
1,mae,1.814703,train,boost_tree,,global
2,mape,2.194239,train,boost_tree,,global
3,smape,2.159085,train,boost_tree,,global
4,r_squared,0.995618,train,boost_tree,,global
5,mda,97.560976,train,boost_tree,,global
6,rmse,34.32853,test,boost_tree,,global
7,mae,27.419104,test,boost_tree,,global
8,mape,20.606137,test,boost_tree,,global
9,smape,24.020992,test,boost_tree,,global


## 1.5 - Decision Tree (Regression)

In [33]:
from py_parsnip import decision_tree

# Create specification - single decision tree
spec_decision_tree = decision_tree(
    tree_depth=10,
    min_n=5
).set_mode('regression')

# Fit model
fit_decision_tree = spec_decision_tree.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_decision_tree = fit_decision_tree.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_decision_tree, coefs_decision_tree, stats_decision_tree = fit_decision_tree.extract_outputs()
display(outputs_decision_tree)
display(coefs_decision_tree)
display(stats_decision_tree)

Model fitted successfully!


,date,actuals,fitted,forecast,split,model,model_group_name,group,residuals
0,2020-04-01,137.65,137.650000,137.65,train,decision_tree,,global,137.650000
1,2020-05-01,113.53,112.556667,113.53,train,decision_tree,,global,113.530000
2,2020-06-01,43.31,43.310000,43.31,train,decision_tree,,global,43.310000
3,2020-07-01,79.77,97.022500,79.77,train,decision_tree,,global,79.770000
4,2020-08-01,48.27,70.086667,48.27,train,decision_tree,,global,48.270000
5,2020-09-01,98.01,70.086667,98.01,train,decision_tree,,global,98.010000
6,2020-10-01,49.77,42.625000,49.77,train,decision_tree,,global,49.770000
7,2020-11-01,35.48,42.625000,35.48,train,decision_tree,,global,35.480000
8,2020-12-01,63.98,70.086667,63.98,train,decision_tree,,global,63.980000
9,2021-01-01,109.76,97.022500,109.76,train,decision_tree,,global,109.760000


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,totaltar,0.217747,NaN,NaN,NaN,NaN,NaN,NaN,decision_tree,,global
1,mean_med_diesel_crack_input1_trade_month_lag2,0.212993,NaN,NaN,NaN,NaN,NaN,NaN,decision_tree,,global
2,mean_nwe_hsfo_crack_trade_month_lag1,0.569259,NaN,NaN,NaN,NaN,NaN,NaN,decision_tree,,global


,metric,value,split,model,model_group_name,group
0,rmse,15.491238,train,decision_tree,,global
1,mae,10.005952,train,decision_tree,,global
2,mape,11.288249,train,decision_tree,,global
3,smape,10.702098,train,decision_tree,,global
4,r_squared,0.806943,train,decision_tree,,global
5,mda,90.243902,train,decision_tree,,global
6,adj_r_squared,0.791702,train,decision_tree,,global
7,rmse,33.195582,test,decision_tree,,global
8,mae,26.651778,test,decision_tree,,global
9,mape,20.117566,test,decision_tree,,global


## 1.6 - Random Forest

In [34]:
from py_parsnip import rand_forest

# Create specification - random forest
spec_rand_forest = rand_forest(
    trees=100,
    mtry=3,
    min_n=5
).set_mode('regression')

# Fit model
fit_rand_forest = spec_rand_forest.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_rand_forest = fit_rand_forest.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_rand_forest, coefs_rand_forest, stats_rand_forest = fit_rand_forest.extract_outputs()
display(outputs_rand_forest)
display(coefs_rand_forest)
display(stats_rand_forest)

Model fitted successfully!


,date,actuals,fitted,forecast,split,model,model_group_name,group,residuals
0,2020-04-01,137.65,119.561032,137.65,train,rand_forest,,global,137.650000
1,2020-05-01,113.53,105.778114,113.53,train,rand_forest,,global,113.530000
2,2020-06-01,43.31,76.789846,43.31,train,rand_forest,,global,43.310000
3,2020-07-01,79.77,84.189391,79.77,train,rand_forest,,global,79.770000
4,2020-08-01,48.27,70.392044,48.27,train,rand_forest,,global,48.270000
5,2020-09-01,98.01,74.151645,98.01,train,rand_forest,,global,98.010000
6,2020-10-01,49.77,56.146088,49.77,train,rand_forest,,global,49.770000
7,2020-11-01,35.48,58.339764,35.48,train,rand_forest,,global,35.480000
8,2020-12-01,63.98,56.091735,63.98,train,rand_forest,,global,63.980000
9,2021-01-01,109.76,93.015498,109.76,train,rand_forest,,global,109.760000


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,totaltar,0.147082,NaN,NaN,NaN,NaN,NaN,NaN,rand_forest,,global
1,mean_med_diesel_crack_input1_trade_month_lag2,0.327253,NaN,NaN,NaN,NaN,NaN,NaN,rand_forest,,global
2,mean_nwe_hsfo_crack_trade_month_lag1,0.525664,NaN,NaN,NaN,NaN,NaN,NaN,rand_forest,,global


,metric,value,split,model,model_group_name,group
0,rmse,18.381228,train,rand_forest,,global
1,mae,14.34127,train,rand_forest,,global
2,mape,19.493462,train,rand_forest,,global
3,smape,16.744421,train,rand_forest,,global
4,r_squared,0.728192,train,rand_forest,,global
5,mda,73.170732,train,rand_forest,,global
6,adj_r_squared,0.706734,train,rand_forest,,global
7,rmse,33.240119,test,rand_forest,,global
8,mae,25.270536,test,rand_forest,,global
9,mape,18.770503,test,rand_forest,,global


## 1.7 - Gradient Boosting Models

### XGBoost Engine

In [35]:
from py_parsnip import boost_tree

# Create specification - XGBoost
spec_xgboost = boost_tree(
    trees=100,
    tree_depth=6,
    learn_rate=0.1
).set_engine('xgboost')

# Fit model
fit_xgboost = spec_xgboost.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_xgboost = fit_xgboost.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_xgboost, coefs_xgboost, stats_xgboost = fit_xgboost.extract_outputs()
display(outputs_xgboost)
display(coefs_xgboost)
display(stats_xgboost)

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,136.217911,137.65,1.432089,train,boost_tree,,global
1,2020-05-01,113.53,113.584618,113.53,-0.054618,train,boost_tree,,global
2,2020-06-01,43.31,44.650463,43.31,-1.340463,train,boost_tree,,global
3,2020-07-01,79.77,79.936256,79.77,-0.166256,train,boost_tree,,global
4,2020-08-01,48.27,48.975697,48.27,-0.705697,train,boost_tree,,global
5,2020-09-01,98.01,97.006821,98.01,1.003179,train,boost_tree,,global
6,2020-10-01,49.77,48.666771,49.77,1.103229,train,boost_tree,,global
7,2020-11-01,35.48,37.079018,35.48,-1.599018,train,boost_tree,,global
8,2020-12-01,63.98,63.986847,63.98,-0.006847,train,boost_tree,,global
9,2021-01-01,109.76,109.312119,109.76,0.447881,train,boost_tree,,global


,variable,importance,model,model_group_name,group
0,mean_nwe_hsfo_crack_trade_month_lag1,0.387399,boost_tree,,global
1,totaltar,0.381204,boost_tree,,global
2,mean_med_diesel_crack_input1_trade_month_lag2,0.231397,boost_tree,,global
3,Intercept,0.000000,boost_tree,,global


,metric,value,split,model,model_group_name,group
0,rmse,0.770095,train,boost_tree,,global
1,mae,0.564497,train,boost_tree,,global
2,mape,0.743748,train,boost_tree,,global
3,smape,0.739005,train,boost_tree,,global
4,r_squared,0.999523,train,boost_tree,,global
5,mda,100.0,train,boost_tree,,global
6,rmse,41.993926,test,boost_tree,,global
7,mae,31.979743,test,boost_tree,,global
8,mape,23.843216,test,boost_tree,,global
9,smape,29.177896,test,boost_tree,,global


### CatBoost Engine

In [36]:
# Create specification - CatBoost
spec_catboost = boost_tree(
    trees=100,
    tree_depth=6,
    learn_rate=0.1
).set_engine('catboost')

# Fit model
fit_catboost = spec_catboost.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_catboost = fit_catboost.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_catboost, coefs_catboost, stats_catboost = fit_catboost.extract_outputs()
display(outputs_catboost)
display(coefs_catboost)
display(stats_catboost)

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,122.319187,137.65,15.330813,train,boost_tree,,global
1,2020-05-01,113.53,111.249924,113.53,2.280076,train,boost_tree,,global
2,2020-06-01,43.31,65.863610,43.31,-22.553610,train,boost_tree,,global
3,2020-07-01,79.77,85.159886,79.77,-5.389886,train,boost_tree,,global
4,2020-08-01,48.27,56.747895,48.27,-8.477895,train,boost_tree,,global
5,2020-09-01,98.01,91.083774,98.01,6.926226,train,boost_tree,,global
6,2020-10-01,49.77,50.377078,49.77,-0.607078,train,boost_tree,,global
7,2020-11-01,35.48,49.769099,35.48,-14.289099,train,boost_tree,,global
8,2020-12-01,63.98,63.475879,63.98,0.504121,train,boost_tree,,global
9,2021-01-01,109.76,97.969280,109.76,11.790720,train,boost_tree,,global


,variable,importance,model,model_group_name,group
0,mean_nwe_hsfo_crack_trade_month_lag1,49.617078,boost_tree,,global
1,mean_med_diesel_crack_input1_trade_month_lag2,42.722072,boost_tree,,global
2,totaltar,7.660850,boost_tree,,global
3,Intercept,0.000000,boost_tree,,global


,metric,value,split,model,model_group_name,group
0,rmse,10.155859,train,boost_tree,,global
1,mae,8.087249,train,boost_tree,,global
2,mape,9.910733,train,boost_tree,,global
3,smape,9.279709,train,boost_tree,,global
4,r_squared,0.917025,train,boost_tree,,global
5,mda,87.804878,train,boost_tree,,global
6,rmse,37.150768,test,boost_tree,,global
7,mae,29.297016,test,boost_tree,,global
8,mape,21.702666,test,boost_tree,,global
9,smape,25.80966,test,boost_tree,,global


## 1.8 - Support Vector Machines

### SVM with RBF Kernel

In [37]:
from py_parsnip import svm_rbf

# Create specification - SVM with RBF kernel
spec_svm_rbf = svm_rbf(
    cost=1.0,
    rbf_sigma=0.1
).set_mode('regression')

# Fit model
fit_svm_rbf = spec_svm_rbf.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_svm_rbf = fit_svm_rbf.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_svm_rbf, coefs_svm_rbf, stats_svm_rbf = fit_svm_rbf.extract_outputs()
display(outputs_svm_rbf)
display(coefs_svm_rbf)
display(stats_svm_rbf)

Model fitted successfully!


,date,actuals,fitted,forecast,split,model,model_group_name,group,residuals
0,2020-04-01,137.65,100.828322,137.65,train,svm_rbf,,global,137.650000
1,2020-05-01,113.53,100.472639,113.53,train,svm_rbf,,global,113.530000
2,2020-06-01,43.31,98.151721,43.31,train,svm_rbf,,global,43.310000
3,2020-07-01,79.77,98.151970,79.77,train,svm_rbf,,global,79.770000
4,2020-08-01,48.27,98.767039,48.27,train,svm_rbf,,global,48.270000
5,2020-09-01,98.01,98.110000,98.01,train,svm_rbf,,global,98.010000
6,2020-10-01,49.77,97.732005,49.77,train,svm_rbf,,global,49.770000
7,2020-11-01,35.48,97.758269,35.48,train,svm_rbf,,global,35.480000
8,2020-12-01,63.98,97.839107,63.98,train,svm_rbf,,global,63.980000
9,2021-01-01,109.76,98.527308,109.76,train,svm_rbf,,global,109.760000


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group


,metric,value,split,model,model_group_name,group
0,rmse,34.362148,train,svm_rbf,,global
1,mae,25.921599,train,svm_rbf,,global
2,mape,37.371245,train,svm_rbf,,global
3,smape,28.193065,train,svm_rbf,,global
4,r_squared,0.050109,train,svm_rbf,,global
5,mda,73.170732,train,svm_rbf,,global
6,adj_r_squared,-0.024882,train,svm_rbf,,global
7,rmse,32.026966,test,svm_rbf,,global
8,mae,25.284836,test,svm_rbf,,global
9,mape,18.450162,test,svm_rbf,,global


### SVM with Linear Kernel

In [38]:
from py_parsnip import svm_linear

# Create specification - SVM with linear kernel
spec_svm_linear = svm_linear(
    cost=1.0
).set_mode('regression')

# Fit model
fit_svm_linear = spec_svm_linear.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_svm_linear = fit_svm_linear.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_svm_linear, coefs_svm_linear, stats_svm_linear = fit_svm_linear.extract_outputs()
display(outputs_svm_linear)
display(coefs_svm_linear)
display(stats_svm_linear)

Model fitted successfully!


/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



,date,actuals,fitted,forecast,split,model,model_group_name,group,residuals
0,2020-04-01,137.65,78.295948,137.65,train,svm_linear,,global,137.650000
1,2020-05-01,113.53,62.424119,113.53,train,svm_linear,,global,113.530000
2,2020-06-01,43.31,49.250677,43.31,train,svm_linear,,global,43.310000
3,2020-07-01,79.77,46.862628,79.77,train,svm_linear,,global,79.770000
4,2020-08-01,48.27,56.811540,48.27,train,svm_linear,,global,48.270000
5,2020-09-01,98.01,59.806480,98.01,train,svm_linear,,global,98.010000
6,2020-10-01,49.77,61.873298,49.77,train,svm_linear,,global,49.770000
7,2020-11-01,35.48,60.947710,35.48,train,svm_linear,,global,35.480000
8,2020-12-01,63.98,58.338592,63.98,train,svm_linear,,global,63.980000
9,2021-01-01,109.76,59.422426,109.76,train,svm_linear,,global,109.760000


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,totaltar,-0.352038,NaN,NaN,NaN,NaN,NaN,NaN,svm_linear,,global
1,mean_med_diesel_crack_input1_trade_month_lag2,-1.057104,NaN,NaN,NaN,NaN,NaN,NaN,svm_linear,,global
2,mean_nwe_hsfo_crack_trade_month_lag1,-0.558698,NaN,NaN,NaN,NaN,NaN,NaN,svm_linear,,global


,metric,value,split,model,model_group_name,group
0,rmse,67.599329,train,svm_linear,,global
1,mae,44.98797,train,svm_linear,,global
2,mape,51.152722,train,svm_linear,,global
3,smape,49.436132,train,svm_linear,,global
4,r_squared,-2.676189,train,svm_linear,,global
5,mda,56.097561,train,svm_linear,,global
6,adj_r_squared,-2.966414,train,svm_linear,,global
7,rmse,44.295779,test,svm_linear,,global
8,mae,35.232687,test,svm_linear,,global
9,mape,26.148142,test,svm_linear,,global


## 1.9 - k-Nearest Neighbors

In [39]:
from py_parsnip import nearest_neighbor

# Create specification - k-NN with k=5
spec_knn_5 = nearest_neighbor(
    neighbors=5,
    weight_func='uniform'
).set_mode('regression')

# Fit model
fit_knn_5 = spec_knn_5.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_knn_5 = fit_knn_5.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_knn_5, coefs_knn_5, stats_knn_5 = fit_knn_5.extract_outputs()
display(outputs_knn_5)
display(coefs_knn_5)
display(stats_knn_5)

Model fitted successfully!


,date,actuals,fitted,forecast,split,model,model_group_name,group,residuals
0,2020-04-01,137.65,119.460,137.65,train,nearest_neighbor,,global,137.650
1,2020-05-01,113.53,81.310,113.53,train,nearest_neighbor,,global,113.530
2,2020-06-01,43.31,75.824,43.31,train,nearest_neighbor,,global,43.310
3,2020-07-01,79.77,75.824,79.77,train,nearest_neighbor,,global,79.770
4,2020-08-01,48.27,71.100,48.27,train,nearest_neighbor,,global,48.270
5,2020-09-01,98.01,71.400,98.01,train,nearest_neighbor,,global,98.010
6,2020-10-01,49.77,71.400,49.77,train,nearest_neighbor,,global,49.770
7,2020-11-01,35.48,71.400,35.48,train,nearest_neighbor,,global,35.480
8,2020-12-01,63.98,71.400,63.98,train,nearest_neighbor,,global,63.980
9,2021-01-01,109.76,71.100,109.76,train,nearest_neighbor,,global,109.760


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group


,metric,value,split,model,model_group_name,group
0,rmse,28.515283,train,nearest_neighbor,,global
1,mae,23.074476,train,nearest_neighbor,,global
2,mape,30.927704,train,nearest_neighbor,,global
3,smape,25.464624,train,nearest_neighbor,,global
4,r_squared,0.345863,train,nearest_neighbor,,global
5,mda,53.658537,train,nearest_neighbor,,global
6,adj_r_squared,0.294221,train,nearest_neighbor,,global
7,rmse,28.37653,test,nearest_neighbor,,global
8,mae,21.162267,test,nearest_neighbor,,global
9,mape,15.435399,test,nearest_neighbor,,global


In [40]:
# k-NN with k=10 and distance weighting
spec_knn_10 = nearest_neighbor(
    neighbors=10,
    weight_func='distance'
).set_mode('regression')

# Fit model
fit_knn_10 = spec_knn_10.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_knn_10 = fit_knn_10.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_knn_10, coefs_knn_10, stats_knn_10 = fit_knn_10.extract_outputs()
display(outputs_knn_10)
display(coefs_knn_10)
display(stats_knn_10)

Model fitted successfully!


,date,actuals,fitted,forecast,split,model,model_group_name,group,residuals
0,2020-04-01,137.65,137.650000,137.65,train,nearest_neighbor,,global,137.650000
1,2020-05-01,113.53,113.530000,113.53,train,nearest_neighbor,,global,113.530000
2,2020-06-01,43.31,43.310000,43.31,train,nearest_neighbor,,global,43.310000
3,2020-07-01,79.77,79.770000,79.77,train,nearest_neighbor,,global,79.770000
4,2020-08-01,48.27,48.270000,48.27,train,nearest_neighbor,,global,48.270000
5,2020-09-01,98.01,98.010000,98.01,train,nearest_neighbor,,global,98.010000
6,2020-10-01,49.77,49.770000,49.77,train,nearest_neighbor,,global,49.770000
7,2020-11-01,35.48,35.480000,35.48,train,nearest_neighbor,,global,35.480000
8,2020-12-01,63.98,63.980000,63.98,train,nearest_neighbor,,global,63.980000
9,2021-01-01,109.76,109.760000,109.76,train,nearest_neighbor,,global,109.760000


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group


,metric,value,split,model,model_group_name,group
0,rmse,0.0,train,nearest_neighbor,,global
1,mae,0.0,train,nearest_neighbor,,global
2,mape,0.0,train,nearest_neighbor,,global
3,smape,0.0,train,nearest_neighbor,,global
4,r_squared,1.0,train,nearest_neighbor,,global
5,mda,100.0,train,nearest_neighbor,,global
6,adj_r_squared,1.0,train,nearest_neighbor,,global
7,rmse,26.069321,test,nearest_neighbor,,global
8,mae,19.559099,test,nearest_neighbor,,global
9,mape,14.166911,test,nearest_neighbor,,global


## 1.10 - Multi-Layer Perceptron (Neural Network)

In [41]:
from py_parsnip import mlp

# Create specification - MLP with single hidden layer
spec_mlp = mlp(
    hidden_units=50,
    epochs=100,
    learn_rate=0.01
).set_mode('regression')

# Fit model
fit_mlp = spec_mlp.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_mlp = fit_mlp.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_mlp, coefs_mlp, stats_mlp = fit_mlp.extract_outputs()
display(outputs_mlp)
display(coefs_mlp)
display(stats_mlp)

Model fitted successfully!


,date,actuals,fitted,forecast,split,model,model_group_name,group,residuals
0,2020-04-01,137.65,92.480627,137.65,train,mlp,,global,137.650000
1,2020-05-01,113.53,70.416477,113.53,train,mlp,,global,113.530000
2,2020-06-01,43.31,52.142173,43.31,train,mlp,,global,43.310000
3,2020-07-01,79.77,48.781877,79.77,train,mlp,,global,79.770000
4,2020-08-01,48.27,62.538820,48.27,train,mlp,,global,48.270000
5,2020-09-01,98.01,66.675856,98.01,train,mlp,,global,98.010000
6,2020-10-01,49.77,69.517460,49.77,train,mlp,,global,49.770000
7,2020-11-01,35.48,68.248564,35.48,train,mlp,,global,35.480000
8,2020-12-01,63.98,64.559566,63.98,train,mlp,,global,63.980000
9,2021-01-01,109.76,66.159663,109.76,train,mlp,,global,109.760000


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,layer_0_to_1,-0.039419,0.232491,NaN,NaN,-0.441093,0.410756,NaN,mlp,,global
1,layer_1_to_2,0.130292,0.223330,NaN,NaN,-0.197719,0.457221,NaN,mlp,,global


,metric,value,split,model,model_group_name,group
0,rmse,37.618409,train,mlp,,global
1,mae,32.406193,train,mlp,,global
2,mape,38.140598,train,mlp,,global
3,smape,36.1172,train,mlp,,global
4,r_squared,-0.13845,train,mlp,,global
5,mda,60.97561,train,mlp,,global
6,adj_r_squared,-0.228327,train,mlp,,global
7,rmse,41.710348,test,mlp,,global
8,mae,35.526535,test,mlp,,global
9,mape,27.049912,test,mlp,,global


## 1.11 - Poisson Regression (Count Data)

In [42]:
from py_parsnip import poisson_reg

# Create specification - Poisson regression
spec_poisson = poisson_reg()

# Fit model
fit_poisson = spec_poisson.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_poisson = fit_poisson.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_poisson, coefs_poisson, stats_poisson = fit_poisson.extract_outputs()
display(outputs_poisson)
display(coefs_poisson)
display(stats_poisson)

Model fitted successfully!


/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1923: FutureWarning:

The bic value is computed using the deviance formula. After 0.13 this will change to the log-likelihood based formula. This change has no impact on the relative rank of models compared using BIC. You can directly access the log-likelihood version using the `bic_llf` attribute. You can suppress this message by calling statsmodels.genmod.generalized_linear_model.SET_USE_BIC_LLF with True to get the LLF-based version now or False to retainthe deviance version.



,date,actuals,fitted,forecast,residuals,split,pearson_resid,deviance_resid,model,model_group_name,group
0,2020-04-01,137.65,102.998129,137.65,34.651871,train,3.414381,3.245643,poisson_reg,,global
1,2020-05-01,113.53,100.417448,113.53,13.112552,train,1.308527,1.281495,poisson_reg,,global
2,2020-06-01,43.31,98.924681,43.31,-55.614681,train,-5.591613,-6.299475,poisson_reg,,global
3,2020-07-01,79.77,97.906372,79.77,-18.136372,train,-1.832926,-1.894391,poisson_reg,,global
4,2020-08-01,48.27,98.345133,48.27,-50.075133,train,-5.049469,-5.607617,poisson_reg,,global
5,2020-09-01,98.01,98.506040,98.01,-0.496040,train,-0.049979,-0.050021,poisson_reg,,global
6,2020-10-01,49.77,98.452021,49.77,-48.682021,train,-4.906325,-5.427904,poisson_reg,,global
7,2020-11-01,35.48,98.521475,35.48,-63.041475,train,-6.351275,-7.321961,poisson_reg,,global
8,2020-12-01,63.98,97.339887,63.98,-33.359887,train,-3.381265,-3.608719,poisson_reg,,global
9,2021-01-01,109.76,98.662261,109.76,11.097739,train,1.117272,1.097251,poisson_reg,,global


,variable,coefficient,std_error,z_stat,p_value,ci_0.025,ci_0.975,model,model_group_name,group
0,Intercept,4.519788,0.028329,159.545534,0.000000e+00,4.464264,4.575312,poisson_reg,,global
1,totaltar,-0.004343,0.000550,-7.900434,2.779348e-15,-0.005420,-0.003265,poisson_reg,,global
2,mean_med_diesel_crack_input1_trade_month_lag2,-0.000978,0.000319,-3.062356,2.196024e-03,-0.001604,-0.000352,poisson_reg,,global
3,mean_nwe_hsfo_crack_trade_month_lag1,-0.005248,0.001358,-3.865312,1.109472e-04,-0.007910,-0.002587,poisson_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,31.924483,train,poisson_reg,,global
1,mae,25.397377,train,poisson_reg,,global
2,mape,31.474191,train,poisson_reg,,global
3,poisson_deviance,10.339147,train,poisson_reg,,global
4,pseudo_r_squared,0.220458,train,poisson_reg,,global
5,rmse,44.098627,test,poisson_reg,,global
6,mae,35.539951,test,poisson_reg,,global
7,mape,26.397142,test,poisson_reg,,global
8,poisson_deviance,19.364109,test,poisson_reg,,global
9,pseudo_r_squared,-5.542449,test,poisson_reg,,global


## 1.12 - MARS (Multivariate Adaptive Regression Splines)

In [43]:
# ! pip install pyearth
# from py_parsnip import mars

# # Create specification - MARS
# spec_mars = mars(
#     num_terms=10,
#     prod_degree=2
# )

# # Fit model
# fit_mars = spec_mars.fit(train_data, FORMULA_STR)

# # Evaluate on test set
# fit_mars = fit_mars.evaluate(test_data)

# print("Model fitted successfully!")

# # Extract outputs
# outputs_mars, coefs_mars, stats_mars = fit_mars.extract_outputs()
# display(outputs_mars)
# display(coefs_mars)
# display(stats_mars)

## 1.13 - Seasonal Decomposition (STL)

In [44]:
# from py_parsnip import seasonal_reg

# # Create specification - STL decomposition
# spec_stl = seasonal_reg(
#     method='stl',
#     seasonal_period=12
# )

# # Fit model
# fit_stl = spec_stl.fit(train_data, FORMULA_STR)

# # Evaluate on test set
# fit_stl = fit_stl.evaluate(test_data)

# print("Model fitted successfully!")

# # Extract outputs
# outputs_stl, coefs_stl, stats_stl = fit_stl.extract_outputs()
# display(outputs_stl)
# display(coefs_stl)
# display(stats_stl)

## 1.14 - ARIMA Variations

### Non-Seasonal ARIMA

In [45]:
from py_parsnip import arima_reg

# Create specification - Non-seasonal ARIMA(1,1,1)
spec_arima_nonseasonal = arima_reg(
    non_seasonal_ar=1,
    non_seasonal_differences=1,
    non_seasonal_ma=1
)

# Fit model
fit_arima_nonseasonal = spec_arima_nonseasonal.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_arima_nonseasonal = fit_arima_nonseasonal.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_arima_ns, coefs_arima_ns, stats_arima_ns = fit_arima_nonseasonal.extract_outputs()
display(outputs_arima_ns)
display(coefs_arima_ns)
display(stats_arima_ns)

Model fitted successfully!


/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.



,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,8.520330,137.65,129.129670,train,arima_reg,,global
1,2020-05-01,113.53,151.280543,113.53,-37.750543,train,arima_reg,,global
2,2020-06-01,43.31,107.131727,43.31,-63.821727,train,arima_reg,,global
3,2020-07-01,79.77,57.632502,79.77,22.137498,train,arima_reg,,global
4,2020-08-01,48.27,74.411735,48.27,-26.141735,train,arima_reg,,global
5,2020-09-01,98.01,60.050334,98.01,37.959666,train,arima_reg,,global
6,2020-10-01,49.77,80.104028,49.77,-30.334028,train,arima_reg,,global
7,2020-11-01,35.48,62.934730,35.48,-27.454730,train,arima_reg,,global
8,2020-12-01,63.98,53.997004,63.98,9.982996,train,arima_reg,,global
9,2021-01-01,109.76,63.619534,109.76,46.140466,train,arima_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,totaltar,-0.295281,0.469192,-0.629339,5.291273e-01,-1.214881,0.624319,NaN,arima_reg,,global
1,mean_med_diesel_crack_input1_trade_month_lag2,-0.069953,0.176029,-0.397397,6.910744e-01,-0.414963,0.275056,NaN,arima_reg,,global
2,mean_nwe_hsfo_crack_trade_month_lag1,-0.401606,0.491615,-0.816911,4.139795e-01,-1.365154,0.561942,NaN,arima_reg,,global
3,ar.L1,0.240145,0.219350,1.094804,2.736024e-01,-0.189773,0.670063,NaN,arima_reg,,global
4,ma.L1,-0.890347,0.135068,-6.591844,4.343974e-11,-1.155075,-0.625619,NaN,arima_reg,,global
5,sigma2,943.098498,269.673101,3.497192,4.701840e-04,414.548932,1471.648064,NaN,arima_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,37.696178,train,arima_reg,,global
1,mae,29.490056,train,arima_reg,,global
2,mape,32.397011,train,arima_reg,,global
3,smape,32.004008,train,arima_reg,,global
4,r_squared,-0.143162,train,arima_reg,,global
5,mda,41.463415,train,arima_reg,,global
6,rmse,41.057878,test,arima_reg,,global
7,mae,33.629256,test,arima_reg,,global
8,mape,25.005416,test,arima_reg,,global
9,smape,30.044407,test,arima_reg,,global


### Auto ARIMA (Automatic Order Selection)

In [46]:
# # Create specification - Auto ARIMA with search constraints
# spec_auto_arima = arima_reg(
#     non_seasonal_ar=5,      # max_p = 5
#     non_seasonal_differences=2,  # max_d = 2
#     non_seasonal_ma=5,      # max_q = 5
#     seasonal_period=12
# ).set_engine('auto_arima')

# # Fit model - will search for best (p,d,q) within constraints
# fit_auto_arima = spec_auto_arima.fit(train_data, FORMULA_STR)

# # Evaluate on test set
# fit_auto_arima = fit_auto_arima.evaluate(test_data)

# print("Model fitted successfully!")
# print(f"Selected order: {fit_auto_arima.fit_data.get('order', 'N/A')}")

# # Extract outputs
# outputs_auto_arima, coefs_auto_arima, stats_auto_arima = fit_auto_arima.extract_outputs()
# display(outputs_auto_arima)
# display(coefs_auto_arima)
# display(stats_auto_arima)

## 1.15 - Exponential Smoothing Variations

### Additive Seasonality

In [47]:
from py_parsnip import exp_smoothing

# ETS(A,A,A) - All additive
spec_ets_aaa = exp_smoothing(
    error='add',
    trend='add',
    season='add',
    seasonal_period=12
)

# Fit model
fit_ets_aaa = spec_ets_aaa.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_ets_aaa = fit_ets_aaa.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_ets_aaa, coefs_ets_aaa, stats_ets_aaa = fit_ets_aaa.extract_outputs()
display(outputs_ets_aaa)
display(coefs_ets_aaa)
display(stats_ets_aaa)

Model fitted successfully!


/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.



,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,119.568266,137.65,18.081734,train,exp_smoothing,,global
1,2020-05-01,113.53,102.034604,113.53,11.495396,train,exp_smoothing,,global
2,2020-06-01,43.31,72.949596,43.31,-29.639596,train,exp_smoothing,,global
3,2020-07-01,79.77,90.679617,79.77,-10.909617,train,exp_smoothing,,global
4,2020-08-01,48.27,78.521668,48.27,-30.251668,train,exp_smoothing,,global
5,2020-09-01,98.01,69.405629,98.01,28.604371,train,exp_smoothing,,global
6,2020-10-01,49.77,61.577041,49.77,-11.807041,train,exp_smoothing,,global
7,2020-11-01,35.48,89.866576,35.48,-54.386576,train,exp_smoothing,,global
8,2020-12-01,63.98,112.668013,63.98,-48.688013,train,exp_smoothing,,global
9,2021-01-01,109.76,94.222961,109.76,15.537039,train,exp_smoothing,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,alpha (smoothing_level),1.490116e-08,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
1,beta (smoothing_trend),0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
2,gamma (smoothing_seasonal),0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
3,phi (damping_trend),NaN,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
4,l0 (initial_level),8.749568e+01,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
5,b0 (initial_trend),4.893895e-01,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global


,metric,value,split,model,model_group_name,group
0,rmse,29.174087,train,exp_smoothing,,global
1,mae,24.100522,train,exp_smoothing,,global
2,mape,32.149773,train,exp_smoothing,,global
3,smape,26.919124,train,exp_smoothing,,global
4,r_squared,0.315289,train,exp_smoothing,,global
5,mda,68.292683,train,exp_smoothing,,global
6,rmse,26.522722,test,exp_smoothing,,global
7,mae,20.040026,test,exp_smoothing,,global
8,mape,15.517037,test,exp_smoothing,,global
9,smape,17.038668,test,exp_smoothing,,global


### Multiplicative Seasonality

In [48]:
# ETS(A,A,M) - Multiplicative seasonality
spec_ets_aam = exp_smoothing(
    error='add',
    trend='add',
    season='mul',
    seasonal_period=12
)

# Fit model
fit_ets_aam = spec_ets_aam.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_ets_aam = fit_ets_aam.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_ets_aam, coefs_ets_aam, stats_ets_aam = fit_ets_aam.extract_outputs()
display(outputs_ets_aam)
display(coefs_ets_aam)
display(stats_ets_aam)

/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.



Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,114.332744,137.65,23.317256,train,exp_smoothing,,global
1,2020-05-01,113.53,97.365827,113.53,16.164173,train,exp_smoothing,,global
2,2020-06-01,43.31,72.870232,43.31,-29.560232,train,exp_smoothing,,global
3,2020-07-01,79.77,89.522937,79.77,-9.752937,train,exp_smoothing,,global
4,2020-08-01,48.27,78.817592,48.27,-30.547592,train,exp_smoothing,,global
5,2020-09-01,98.01,67.767290,98.01,30.242710,train,exp_smoothing,,global
6,2020-10-01,49.77,62.000122,49.77,-12.230122,train,exp_smoothing,,global
7,2020-11-01,35.48,90.453272,35.48,-54.973272,train,exp_smoothing,,global
8,2020-12-01,63.98,110.830834,63.98,-46.850834,train,exp_smoothing,,global
9,2021-01-01,109.76,92.116482,109.76,17.643518,train,exp_smoothing,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,alpha (smoothing_level),1.490116e-08,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
1,beta (smoothing_trend),0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
2,gamma (smoothing_seasonal),0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
3,phi (damping_trend),NaN,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
4,l0 (initial_level),7.300620e+01,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
5,b0 (initial_trend),5.023695e-01,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global


,metric,value,split,model,model_group_name,group
0,rmse,28.895677,train,exp_smoothing,,global
1,mae,24.030276,train,exp_smoothing,,global
2,mape,31.889076,train,exp_smoothing,,global
3,smape,26.904127,train,exp_smoothing,,global
4,r_squared,0.328295,train,exp_smoothing,,global
5,mda,65.853659,train,exp_smoothing,,global
6,rmse,27.601093,test,exp_smoothing,,global
7,mae,20.430424,test,exp_smoothing,,global
8,mape,16.022215,test,exp_smoothing,,global
9,smape,17.427549,test,exp_smoothing,,global


## 1.16 - Prophet Variations

### High Flexibility

In [49]:
from py_parsnip import prophet_reg

# Prophet with high flexibility
spec_prophet_flexible = prophet_reg(
    n_changepoints=50,
    changepoint_prior_scale=0.5,  # High flexibility
    seasonality_prior_scale=10.0
)

# Fit model
fit_prophet_flex = spec_prophet_flexible.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_prophet_flex = fit_prophet_flex.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_prophet_flex, coefs_prophet_flex, stats_prophet_flex = fit_prophet_flex.extract_outputs()
display(outputs_prophet_flex)
display(coefs_prophet_flex)
display(stats_prophet_flex)

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,105.355195,137.65,32.294805,train,prophet_reg,,global
1,2020-05-01,113.53,109.369254,113.53,4.160746,train,prophet_reg,,global
2,2020-06-01,43.31,59.367296,43.31,-16.057296,train,prophet_reg,,global
3,2020-07-01,79.77,62.577728,79.77,17.192272,train,prophet_reg,,global
4,2020-08-01,48.27,55.263113,48.27,-6.993113,train,prophet_reg,,global
5,2020-09-01,98.01,110.237698,98.01,-12.227698,train,prophet_reg,,global
6,2020-10-01,49.77,74.694060,49.77,-24.924060,train,prophet_reg,,global
7,2020-11-01,35.48,55.538488,35.48,-20.058488,train,prophet_reg,,global
8,2020-12-01,63.98,97.328947,63.98,-33.348947,train,prophet_reg,,global
9,2021-01-01,109.76,113.783269,109.76,-4.023269,train,prophet_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,growth,linear,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
1,changepoint_prior_scale,0.5,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
2,seasonality_prior_scale,10.0,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
3,seasonality_mode,additive,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
4,n_changepoints,32,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,20.284405,train,prophet_reg,,global
1,mae,16.080761,train,prophet_reg,,global
2,mape,17.950316,train,prophet_reg,,global
3,smape,16.864417,train,prophet_reg,,global
4,r_squared,0.668993,train,prophet_reg,,global
5,mda,80.487805,train,prophet_reg,,global
6,rmse,52.316949,test,prophet_reg,,global
7,mae,45.606419,test,prophet_reg,,global
8,mape,35.683566,test,prophet_reg,,global
9,smape,43.26619,test,prophet_reg,,global


### Low Flexibility (Conservative)

In [50]:
# Prophet with low flexibility
spec_prophet_conservative = prophet_reg(
    n_changepoints=10,
    changepoint_prior_scale=0.01,  # Low flexibility
    seasonality_prior_scale=1.0
)

# Fit model
fit_prophet_cons = spec_prophet_conservative.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_prophet_cons = fit_prophet_cons.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs
outputs_prophet_cons, coefs_prophet_cons, stats_prophet_cons = fit_prophet_cons.extract_outputs()
display(outputs_prophet_cons)
display(coefs_prophet_cons)
display(stats_prophet_cons)

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,113.366503,137.65,24.283497,train,prophet_reg,,global
1,2020-05-01,113.53,112.789842,113.53,0.740158,train,prophet_reg,,global
2,2020-06-01,43.31,68.487441,43.31,-25.177441,train,prophet_reg,,global
3,2020-07-01,79.77,75.100474,79.77,4.669526,train,prophet_reg,,global
4,2020-08-01,48.27,67.536181,48.27,-19.266181,train,prophet_reg,,global
5,2020-09-01,98.01,107.782995,98.01,-9.772995,train,prophet_reg,,global
6,2020-10-01,49.77,75.340419,49.77,-25.570419,train,prophet_reg,,global
7,2020-11-01,35.48,75.100217,35.48,-39.620217,train,prophet_reg,,global
8,2020-12-01,63.98,105.057950,63.98,-41.077950,train,prophet_reg,,global
9,2021-01-01,109.76,104.075346,109.76,5.684654,train,prophet_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,growth,linear,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
1,changepoint_prior_scale,0.01,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
2,seasonality_prior_scale,1.0,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
3,seasonality_mode,additive,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
4,n_changepoints,10,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,22.668391,train,prophet_reg,,global
1,mae,18.407255,train,prophet_reg,,global
2,mape,21.961579,train,prophet_reg,,global
3,smape,19.843098,train,prophet_reg,,global
4,r_squared,0.586615,train,prophet_reg,,global
5,mda,78.04878,train,prophet_reg,,global
6,rmse,34.899007,test,prophet_reg,,global
7,mae,28.851545,test,prophet_reg,,global
8,mape,22.241627,test,prophet_reg,,global
9,smape,24.109859,test,prophet_reg,,global


## 1.17 - Recursive Forecasting (Multi-Step Ahead)

In [51]:
from py_parsnip import recursive_reg, rand_forest

# Create specification - Recursive forecasting with Random Forest
spec_recursive = recursive_reg(
    base_model=rand_forest().set_mode('regression'),
    lags=[1, 2, 3, 6, 12],  # Use specific lags
    differentiation=None
)

# Fit model - requires date column as index
train_indexed = train_data.set_index('date')
fit_recursive = spec_recursive.fit(train_indexed, FORMULA_STR)

# Evaluate on test set
test_indexed = test_data.set_index('date')
fit_recursive = fit_recursive.evaluate(test_indexed)

print("Model fitted successfully!")

# Extract outputs
outputs_recursive, coefs_recursive, stats_recursive = fit_recursive.extract_outputs()
display(outputs_recursive)
display(coefs_recursive)
display(stats_recursive)

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,NaN,137.65,NaN,train,recursive_reg,,global
1,2020-05-01,113.53,NaN,113.53,NaN,train,recursive_reg,,global
2,2020-06-01,43.31,NaN,43.31,NaN,train,recursive_reg,,global
3,2020-07-01,79.77,NaN,79.77,NaN,train,recursive_reg,,global
4,2020-08-01,48.27,NaN,48.27,NaN,train,recursive_reg,,global
5,2020-09-01,98.01,NaN,98.01,NaN,train,recursive_reg,,global
6,2020-10-01,49.77,NaN,49.77,NaN,train,recursive_reg,,global
7,2020-11-01,35.48,NaN,35.48,NaN,train,recursive_reg,,global
8,2020-12-01,63.98,NaN,63.98,NaN,train,recursive_reg,,global
9,2021-01-01,109.76,NaN,109.76,NaN,train,recursive_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,lag_1,0.069053,NaN,NaN,NaN,NaN,NaN,NaN,recursive_reg,,global
1,lag_2,0.237229,NaN,NaN,NaN,NaN,NaN,NaN,recursive_reg,,global
2,lag_3,0.027429,NaN,NaN,NaN,NaN,NaN,NaN,recursive_reg,,global
3,lag_6,0.229324,NaN,NaN,NaN,NaN,NaN,NaN,recursive_reg,,global
4,lag_12,0.046417,NaN,NaN,NaN,NaN,NaN,NaN,recursive_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,NaN,train,recursive_reg,,global
1,mae,NaN,train,recursive_reg,,global
2,mape,NaN,train,recursive_reg,,global
3,r_squared,NaN,train,recursive_reg,,global
4,rmse,25.064719,test,recursive_reg,,global
5,mae,18.185684,test,recursive_reg,,global
6,mape,13.053908,test,recursive_reg,,global
7,r_squared,-0.624963,test,recursive_reg,,global
8,formula,target ~ totaltar +mean_med_diesel_crack_input...,,recursive_reg,,global
9,n_obs_train,42,train,recursive_reg,,global


Recipe

In [52]:

# IMPUTATION (6 steps):
#   step_impute_median()   - Fill missing with median
#   step_impute_mean()     - Fill missing with mean
#   step_impute_mode()     - Fill missing with mode
#   step_impute_knn()      - KNN imputation
#   step_impute_bag()      - Bagged tree imputation
#   step_impute_linear()   - Linear model imputation

# NORMALIZATION (4 steps):
#   step_normalize()       - Z-score normalization
#   step_range()           - Min-max scaling [0,1]
#   step_center()          - Center to mean=0
#   step_scale()           - Scale to sd=1

# ENCODING (6 steps):
#   step_dummy()           - One-hot encoding
#   step_one_hot()         - One-hot (alternative)
#   step_target_encode()   - Target encoding
#   step_ordinal()         - Ordinal encoding
#   step_bin()             - Discretization
#   step_date()            - Extract date features

# FEATURE ENGINEERING (8 steps):
#   step_poly()            - Polynomial features
#   step_interact()        - Interaction terms
#   step_ns()              - Natural splines
#   step_bs()              - B-splines
#   step_pca()             - Principal components
#   step_log()             - Log transform
#   step_sqrt()            - Square root transform
#   step_inverse()         - Inverse transform

# FILTERING (6 steps):
#   step_corr()            - Remove correlated features
#   step_nzv()             - Remove zero-variance
#   step_filter_missing()  - Remove columns with too many NAs
#   step_select()          - Select specific columns
#   step_rm()              - Remove columns
#   step_filter()          - Row filtering

# ROW OPERATIONS (6 steps):
#   step_sample()          - Sample rows
#   step_filter()          - Filter rows by condition
#   step_slice()           - Select row ranges
#   step_arrange()         - Sort rows
#   step_shuffle()         - Shuffle rows
#   step_lag()             - Create lagged features

# TRANSFORMATIONS (6 steps):
#   step_mutate()          - Create/modify columns
#   step_discretize()      - Discretization
#   step_cut()             - Cut into bins
#   step_BoxCox()          - Box-Cox transform
#   step_YeoJohnson()      - Yeo-Johnson transform
#   step_other()           - Pool infrequent categories

# TIME SERIES (4 steps):
#   step_lag()             - Create lags
#   step_diff()            - Differencing
#   step_date()            - Extract date features
#   step_timeseries_signature() - Comprehensive time features


In [53]:
'Create a simple recipe'

rec = (
    recipe()
    .step_normalize()  # Normalize numeric columns
    
)
# Fit on training data
rec_fit = rec.prep(train_data)

# Apply to training data
train_processed = rec_fit.bake(train_data)

print("\nProcessed training data:")

# Apply to test data (using training parameters!)
test_processed = rec_fit.bake(test_data)
print("\nProcessed test data:")
display(train_processed)
display(test_processed)



Processed training data:

Processed test data:


,date,mean_med_diesel_crack_input1_trade_month_lag2,mean_nwe_hsfo_crack_trade_month_lag1,mean_nwe_lsfo_crack_trade_month,mean_nwe_ulsfo_crack_trade_month_lag3,mean_sing_gasoline_vs_vlsfo_trade_month,mean_sing_vlsfo_crack_trade_month_lag3,new_sweet_sr_margin,target,totaltar
0,2020-04-01,-0.088770,-0.094595,0.046119,1.487556,-2.378036,2.031849,0.358746,1.096099,-0.237232
1,2020-05-01,0.172544,0.089438,1.195442,0.793182,-0.956278,1.074411,0.389075,0.411977,-0.237232
2,2020-06-01,0.402280,0.137482,-0.014980,-0.283450,-0.090439,-0.246999,2.220933,-1.579693,-0.237232
3,2020-07-01,0.427807,0.277542,0.105567,-0.597906,0.118757,-0.654823,-0.253895,-0.545568,-0.237232
4,2020-08-01,0.239857,0.358972,0.118778,-1.317998,-0.297698,-0.986637,-0.602676,-1.439011,-0.237232
5,2020-09-01,0.183889,0.378516,0.209601,-0.954445,0.107135,-1.002717,-0.420703,-0.028221,-0.237232
6,2020-10-01,0.141723,0.420859,0.579498,-0.907686,-0.096250,-1.053877,-0.796780,-1.396466,-0.237232
7,2020-11-01,0.161577,0.393987,0.850315,-0.934573,-0.694783,-0.879931,-0.551117,-1.801778,-0.237232
8,2020-12-01,0.187859,0.560105,0.708301,-1.072512,-0.743208,-0.992484,0.216201,-0.993424,-0.237232
9,2021-01-01,0.194855,0.345129,0.657110,-0.839885,-0.747082,-0.700137,0.201036,0.305047,-0.237232


,date,mean_med_diesel_crack_input1_trade_month_lag2,mean_nwe_hsfo_crack_trade_month_lag1,mean_nwe_lsfo_crack_trade_month,mean_nwe_ulsfo_crack_trade_month_lag3,mean_sing_gasoline_vs_vlsfo_trade_month,mean_sing_vlsfo_crack_trade_month_lag3,new_sweet_sr_margin,target,totaltar
0,2023-10-01,-0.603078,0.495775,-0.521937,-0.600244,-0.260895,-0.252846,1.486976,0.245201,-0.237232
1,2023-11-01,-0.680981,0.024293,-0.589642,-0.463473,-0.853617,-0.238228,0.813678,0.549255,-0.237232
2,2023-12-01,-0.685519,-0.290842,-0.553312,-0.851575,1.366184,-0.413637,-1.078838,0.397795,-0.237232
3,2024-01-01,-0.589653,-0.052251,-0.350199,-0.721818,0.426739,-0.227996,0.234398,0.670650,-0.237232
4,2024-02-01,-0.479418,1.820648,-0.738261,-0.322026,0.878060,0.545263,1.080571,0.853309,-0.237232
5,2024-03-01,-0.467694,-0.179283,-0.578082,-0.099920,0.568140,0.080430,0.725724,1.320736,0.432258
6,2024-04-01,-0.509860,1.814133,-0.802663,-0.315013,1.211224,-0.156371,0.774250,1.679816,0.983528
7,2024-05-01,-0.574149,-0.285142,-0.829084,0.344293,0.637872,0.179828,0.574080,0.142809,0.050824
8,2024-06-01,-0.677388,-0.042480,-0.814222,0.216874,0.064521,0.138900,-0.763419,2.105265,0.983528
9,2024-07-01,-0.604402,0.142367,-0.612760,0.034513,-0.028455,-0.327394,-0.017331,0.771340,1.244599


## 2.1 - Workflows

In [54]:
# FORMULA APPROACH

# Simple formula-based workflow
# Limitations: Can't easily normalize, encode categoricals, etc.
wf_formula = (
    workflow()
    .add_formula(FORMULA_STR)
    .add_model(linear_reg())
)

fit_formula = wf_formula.fit(train_processed)
fit_formula = fit_formula.evaluate(test_processed)

outputs_formula, coefs_formula, stats_formula = fit_formula.extract_outputs()
display(outputs_formula)
display(coefs_formula)
display(stats_formula)


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,1.096099,0.114195,1.096099,0.981904,train,linear_reg,,global
1,2020-05-01,0.411977,0.047536,0.411977,0.364441,train,linear_reg,,global
2,2020-06-01,-1.579693,0.007593,-1.579693,-1.587286,train,linear_reg,,global
3,2020-07-01,-0.545568,-0.018947,-0.545568,-0.526621,train,linear_reg,,global
4,2020-08-01,-1.439011,-0.006077,-1.439011,-1.432935,train,linear_reg,,global
5,2020-09-01,-0.028221,-0.001472,-0.028221,-0.026749,train,linear_reg,,global
6,2020-10-01,-1.396466,-0.002534,-1.396466,-1.393932,train,linear_reg,,global
7,2020-11-01,-1.801778,-0.000897,-1.801778,-1.800881,train,linear_reg,,global
8,2020-12-01,-0.993424,-0.031817,-0.993424,-0.961607,train,linear_reg,,global
9,2021-01-01,0.305047,0.002475,0.305047,0.302572,train,linear_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,1.323479e-16,0.146191,9.053078e-16,1.000000,-0.295948,0.295948,NaN,linear_reg,,global
1,totaltar,-3.637268e-01,0.146552,-2.481899e+00,0.017605,-0.660406,-0.067048,1.004942,linear_reg,,global
2,mean_med_diesel_crack_input1_trade_month_lag2,-1.395542e-01,0.146915,-9.498981e-01,0.348167,-0.436968,0.157860,1.009927,linear_reg,,global
3,mean_nwe_hsfo_crack_trade_month_lag1,-1.640550e-01,0.146573,-1.119275e+00,0.270048,-0.460776,0.132666,1.005225,linear_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,0.901182,train,linear_reg,,global
1,mae,0.710086,train,linear_reg,,global
2,mape,223.87133,train,linear_reg,,global
3,smape,163.029273,train,linear_reg,,global
4,r_squared,0.18787,train,linear_reg,,global
5,mda,58.536585,train,linear_reg,,global
6,adj_r_squared,0.123755,train,linear_reg,,global
7,rmse,1.123305,test,linear_reg,,global
8,mae,0.888622,test,linear_reg,,global
9,mape,147.170781,test,linear_reg,,global


In [55]:
# from py_recipes.selectors import (
#     all_numeric,      # All numeric columns
#     all_nominal,      # All categorical columns
#     all_predictors,   # All predictor columns
#     all_outcomes,     # All outcome columns
#     has_role,         # Columns with specific role
#     has_type,         # Columns with specific type
# )

print("APPROACH 2: Recipe-Based Workflow (Powerful)")

# Recipe-based workflow with advanced feature engineering
# Advantages: Normalization, encoding, imputation, feature creation, etc.

# For recipes, we prep/bake outside the workflow, then use formula in workflow
rec = (
    recipe()  # Create empty recipe
    .step_normalize()  # Normalize numeric features (z-score) - None = all numeric
)

rec = (
    recipe()
    .step_impute_median()      # 1. Impute
    #.step_boxcox(contains("crack"))       # 2. Transform
    .step_boxcox()       # 2. Transform
    #.step_normalize()          # 3. Normalize
    #.step_range()          # 3. Normalize
    #.step_center()          # 3. Normalize
    #.step_scale()          # 3. Normalize
    #.step_dummy(["category"])  # 4. Encode
    #.step_corr(threshold=0.9)  # 5. Filter
    #.step_poly(['totaltar'], degree=2)
    #.step_interact([('totaltar', 'mean_med_diesel_crack_input1_trade_month_lag2')])
    #.step_ns('mean_med_diesel_crack_input1_trade_month_lag2', df = 2)
    #.step_pca(["totaltar", "v", "mean_nwe_hsfo_crack_trade_month_lag1"], num_comp=3)
    #.step_nzv()
    #.step_filter_missing(threshold=0.5) # Remove columns with >50% missing
    #.step_lag("mean_med_diesel_crack_input1_trade_month_lag2", lags=[1, 7, 14, 28])  # Lagged features
    #.step_diff(["mean_med_diesel_crack_input1_trade_month_lag2"], lag=1)                # First difference
    #.step_timeseries_signature("date")        # Extract time features
    .step_select_corr(outcome='target', threshold=0.9, method='multicollinearity')  # Remove highly correlated
)

# Prep the recipe on training data
rec_prepped = rec.prep(train_data)

# Apply to both train and test
train_processed_recipe = rec_prepped.bake(train_data)
test_processed_recipe = rec_prepped.bake(test_data)

print(train_processed_recipe.columns)

wf_recipe = (
    workflow()
    # Formula on preprocessed data
    #.add_formula(FORMULA_STR)
    .add_recipe(rec)
    .add_model(linear_reg())
)

fit_recipe = wf_recipe.fit(train_data)
fit_recipe = fit_recipe.evaluate(test_data)

outputs_recipe, coefs_recipe, stats_recipe = fit_recipe.extract_outputs()
display(outputs_recipe)
display(coefs_recipe)
display(stats_recipe)

APPROACH 2: Recipe-Based Workflow (Powerful)
Index(['date', 'mean_med_diesel_crack_input1_trade_month_lag2',
       'mean_nwe_hsfo_crack_trade_month_lag1',
       'mean_nwe_lsfo_crack_trade_month',
       'mean_nwe_ulsfo_crack_trade_month_lag3',
       'mean_sing_gasoline_vs_vlsfo_trade_month', 'new_sweet_sr_margin',
       'target', 'totaltar'],
      dtype='object')


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,80.440353,61.045674,80.440353,19.394679,train,linear_reg,,global
1,2020-05-01,67.926466,61.655635,67.926466,6.270830,train,linear_reg,,global
2,2020-06-01,28.882702,61.934255,28.882702,-33.051552,train,linear_reg,,global
3,2020-07-01,49.766772,57.931067,49.766772,-8.164295,train,linear_reg,,global
4,2020-08-01,31.833518,45.094697,31.833518,-13.261179,train,linear_reg,,global
5,2020-09-01,59.685178,53.759895,59.685178,5.925284,train,linear_reg,,global
6,2020-10-01,32.717617,52.878267,32.717617,-20.160651,train,linear_reg,,global
7,2020-11-01,24.127453,50.145055,24.127453,-26.017602,train,linear_reg,,global
8,2020-12-01,40.928859,48.264781,40.928859,-7.335922,train,linear_reg,,global
9,2021-01-01,65.939461,52.026295,65.939461,13.913166,train,linear_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,19.124256,27.406301,0.697805,0.490044,-36.572048,74.820560,NaN,linear_reg,,global
1,mean_med_diesel_crack_input1_trade_month_lag2,-0.032158,0.057680,-0.557526,0.580821,-0.149379,0.085062,1.222203,linear_reg,,global
2,mean_nwe_hsfo_crack_trade_month_lag1,-0.138910,0.241495,-0.575210,0.568937,-0.629688,0.351867,1.155154,linear_reg,,global
3,mean_nwe_lsfo_crack_trade_month,-0.233839,0.542270,-0.431222,0.669029,-1.335863,0.868186,1.416318,linear_reg,,global
4,mean_nwe_ulsfo_crack_trade_month_lag3,22.070911,14.922514,1.479034,0.148337,-8.255287,52.397109,1.591220,linear_reg,,global
5,mean_sing_gasoline_vs_vlsfo_trade_month,0.702057,0.631326,1.112035,0.273925,-0.580952,1.985066,1.395226,linear_reg,,global
6,new_sweet_sr_margin,0.215382,1.130854,0.190460,0.850081,-2.082790,2.513555,1.825994,linear_reg,,global
7,totaltar,-0.183634,0.082476,-2.226529,0.032709,-0.351245,-0.016024,1.947766,linear_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,16.089461,train,linear_reg,,global
1,mae,13.080208,train,linear_reg,,global
2,mape,25.354143,train,linear_reg,,global
3,smape,22.677264,train,linear_reg,,global
4,r_squared,0.286434,train,linear_reg,,global
5,mda,58.536585,train,linear_reg,,global
6,adj_r_squared,0.139523,train,linear_reg,,global
7,rmse,21.658552,test,linear_reg,,global
8,mae,18.53875,test,linear_reg,,global
9,mape,23.987486,test,linear_reg,,global


In [56]:
print("\n" + "="*70)
print("APPROACH 3: Recipe Without Workflow (Manual prep/bake)")
print("="*70)

# You can also use recipes without workflows - just prep/bake directly
rec_manual = (
    recipe()
    .step_normalize()
)

# Prep on training data
rec_manual_prepped = rec_manual.prep(train_data)

# Bake both datasets
train_manual = rec_manual_prepped.bake(train_data)
test_manual = rec_manual_prepped.bake(test_data)

spec_manual = linear_reg()
fit_manual = spec_manual.fit(train_manual, "target ~ . -date")
fit_manual = fit_manual.evaluate(test_manual)

outputs_manual, coefs_manual, stats_manual = fit_manual.extract_outputs()
display(outputs_manual)
display(coefs_manual)
display(stats_manual)



APPROACH 3: Recipe Without Workflow (Manual prep/bake)


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,1.096099,0.039546,1.096099,1.056553,train,linear_reg,,global
1,2020-05-01,0.411977,-0.016712,0.411977,0.428689,train,linear_reg,,global
2,2020-06-01,-1.579693,0.094006,-1.579693,-1.673700,train,linear_reg,,global
3,2020-07-01,-0.545568,-0.097141,-0.545568,-0.448427,train,linear_reg,,global
4,2020-08-01,-1.439011,-0.281262,-1.439011,-1.157749,train,linear_reg,,global
5,2020-09-01,-0.028221,-0.178389,-0.028221,0.150168,train,linear_reg,,global
6,2020-10-01,-1.396466,-0.282384,-1.396466,-1.114082,train,linear_reg,,global
7,2020-11-01,-1.801778,-0.393669,-1.801778,-1.408109,train,linear_reg,,global
8,2020-12-01,-0.993424,-0.395386,-0.993424,-0.598038,train,linear_reg,,global
9,2021-01-01,0.305047,-0.312078,0.305047,0.617125,train,linear_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,2.628306e-16,0.151184,1.738479e-15,1.000000,-0.307587,0.307587,NaN,linear_reg,,global
1,mean_med_diesel_crack_input1_trade_month_lag2,-9.410508e-02,0.168384,-5.588728e-01,0.580022,-0.436684,0.248474,1.240473,linear_reg,,global
2,mean_nwe_hsfo_crack_trade_month_lag1,-1.121007e-01,0.172873,-6.484566e-01,0.521177,-0.463814,0.239612,1.307501,linear_reg,,global
3,mean_nwe_lsfo_crack_trade_month,-1.038090e-01,0.198654,-5.225630e-01,0.604769,-0.507973,0.300355,1.726553,linear_reg,,global
4,mean_nwe_ulsfo_crack_trade_month_lag3,5.714816e-02,0.655382,8.719818e-02,0.931041,-1.276238,1.390534,18.792161,linear_reg,,global
5,mean_sing_gasoline_vs_vlsfo_trade_month,2.015905e-01,0.192450,1.047494e+00,0.302484,-0.189953,0.593133,1.620407,linear_reg,,global
6,mean_sing_vlsfo_crack_trade_month_lag3,1.464219e-01,0.653304,2.241250e-01,0.824042,-1.182736,1.475580,18.673177,linear_reg,,global
7,new_sweet_sr_margin,5.051546e-02,0.213297,2.368318e-01,0.814251,-0.383440,0.484471,1.990470,linear_reg,,global
8,totaltar,-4.389148e-01,0.250863,-1.749618e+00,0.089480,-0.949300,0.071470,2.753345,linear_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,0.868487,train,linear_reg,,global
1,mae,0.708766,train,linear_reg,,global
2,mape,267.502038,train,linear_reg,,global
3,smape,150.817641,train,linear_reg,,global
4,r_squared,0.24573,train,linear_reg,,global
5,mda,58.536585,train,linear_reg,,global
6,adj_r_squared,0.062877,train,linear_reg,,global
7,rmse,1.154199,test,linear_reg,,global
8,mae,0.943515,test,linear_reg,,global
9,mape,211.503325,test,linear_reg,,global


In [57]:
# method chaining
# Full pipeline in one chain
outputs2, coefficients2, stats2 = (
    workflow()
    .add_formula(FORMULA_STR)
    .add_model(linear_reg().set_engine("sklearn"))
    .fit(train_data)
    .evaluate(test_data)
    .extract_outputs()
)

print("Method chaining complete!")
print(f"\nTest R²: {stats2[(stats2['metric'] == 'r_squared') & (stats2['split'] == 'test')]['value'].values[0]:.4f}")

Method chaining complete!

Test R²: -3.0570


In [58]:
# # Extract components
# model_fit = wf_recipe.extract_fit_parsnip()
# preprocessor = wf_recipe.extract_preprocessor()
# model_spec = wf_recipe.extract_spec_parsnip()

# print("Extracted components:")
# print(f"  Model fit type: {type(model_fit)}")
# print(f"  Preprocessor: {preprocessor}")
# print(f"  Model spec: {model_spec.mode} with {model_spec.engine} engine")

# # Access underlying sklearn model
# sklearn_model = model_fit.fit_data["model"]
# print(f"\n  Underlying model type: {type(sklearn_model)}")
# print(f"  Number of coefficients: {len(sklearn_model.coef_)}")

In [59]:
# Create recipe
rec_for_model = (
    recipe()
    .step_impute_mean()
    .step_normalize()
    #.step_remove("date")
)

# Create workflow
wf = (
    workflow()
    .add_recipe(rec_for_model)
    .add_model(linear_reg().set_engine("sklearn"))
)

print("Workflow created with recipe preprocessing")
print(f"Preprocessor: {type(wf.preprocessor).__name__}")
print(f"Model: {wf.spec.model_type}")

# Fit workflow (recipe is automatically prepped)
wf_fit = wf.fit(train_data)

print("Workflow fitted successfully")
print(f"Preprocessor type: {type(wf_fit.pre).__name__}")

Workflow created with recipe preprocessing
Preprocessor: Recipe
Model: linear_reg
Workflow fitted successfully
Preprocessor type: PreparedRecipe


In [60]:
# Define different formulas (preprocessing strategies)
formulas = [
    "target ~ mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1",                                                        # Minimal model
    "target ~ mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1 + mean_nwe_lsfo_crack_trade_month",                      # Add more features
    "target ~ mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1 + mean_nwe_lsfo_crack_trade_month + totaltar",           # All features
    "target ~ mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1 + mean_nwe_lsfo_crack_trade_month + I(mean_med_diesel_crack_input1_trade_month_lag2*mean_nwe_hsfo_crack_trade_month_lag1)",           # With interaction
    "target ~ mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1 + mean_nwe_lsfo_crack_trade_month + I(mean_med_diesel_crack_input1_trade_month_lag2**2) + I(mean_nwe_hsfo_crack_trade_month_lag1**2)" # With polynomials
]

# Descriptive IDs for each formula
formula_ids = [
    "minimal",
    "medium",
    "full",
    "interaction",
    "polynomial"
]

print(f"Number of preprocessing strategies: {len(formulas)}")

Number of preprocessing strategies: 5
